In [1]:
import pandas as pd
import re
import numpy as np
import os

### Aggregated Total Survey Data

In [2]:
total_2015 = pd.read_csv('../data/survey_csv/2015_total.csv')

In [3]:
total_2015.head()

,DBN,School Type,School Name,Total Parent \nResponse Rate,Total Teacher Response Rate,Total Student Response Rate,Percent Satisfaction: Collaborative Teachers,Percent Satisfaction: Effective School Leadership,Percent Satisfaction: Rigorous Instruction,Percent Satisfaction: Strong Family-Community Ties,Percent Satisfaction: Supportive Environment,Percent Satisfaction: Trust,Percent Satisfaction: Parent,Percent Satisfaction: Student,Percent Satisfaction: Teacher
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01M015,Elementary School,P.S. 015 Roberto Clemente,62.0,100.0,NaN,94.0,91.0,89.0,85.0,86.0,94.0,91.0,NaN,91.0
3,01M019,Elementary School,P.S. 019 Asher Levy,100.0,98.0,NaN,96.0,90.0,96.0,86.0,97.0,94.0,92.0,NaN,96.0
4,01M020,Elementary School,P.S. 020 Anna Silver,56.0,72.0,NaN,77.0,61.0,87.0,80.0,82.0,79.0,86.0,NaN,75.0


In [4]:
# because not all surveys have the parent specific columns in the total dataframe, I am dropping those columns
total_2015 = total_2015[total_2015.columns[:-3]]

In [5]:
total_2015

,DBN,School Type,School Name,Total Parent \nResponse Rate,Total Teacher Response Rate,Total Student Response Rate,Percent Satisfaction: Collaborative Teachers,Percent Satisfaction: Effective School Leadership,Percent Satisfaction: Rigorous Instruction,Percent Satisfaction: Strong Family-Community Ties,Percent Satisfaction: Supportive Environment,Percent Satisfaction: Trust
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01M015,Elementary School,P.S. 015 Roberto Clemente,62.0,100.0,NaN,94.00,91.00,89.00,85.00,86.00,94.00
3,01M019,Elementary School,P.S. 019 Asher Levy,100.0,98.0,NaN,96.00,90.00,96.00,86.00,97.00,94.00
4,01M020,Elementary School,P.S. 020 Anna Silver,56.0,72.0,NaN,77.00,61.00,87.00,80.00,82.00,79.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1858,84K979,High School,Uncommon Charter HS 2,NaN,NaN,NaN,0.90,0.85,0.90,0.83,0.91,0.91
1859,84K989,High School,Achievement First University Prep,NaN,NaN,NaN,0.92,0.85,0.92,0.88,0.90,0.91
1860,84K999,High School,Uncommon Charter HS,NaN,NaN,NaN,0.90,0.87,0.89,0.81,0.87,0.92
1861,84M888,High School,Harlem Village Academy HS,NaN,NaN,NaN,0.88,0.85,0.78,0.74,0.77,0.90


In [6]:
total_cols = [
    'dbn', 'school_type',
    'school_name',
    'parent_response_rate', 
    'teacher_response_rate', 
    'student_response_rate',
    'collab_teachers',
    'leadership',
    'instruction',
    'ties',
    'support',
    'trust'
]

In [7]:
# rename columns
total_2015.columns = total_cols

# drop the first 2 rows with nulls
total_2015.dropna(how='all', axis=0, inplace=True)

# reset index
total_2015.reset_index(drop=True, inplace=True)

In [8]:
# I will assume that all missing data are because of no response, so percent response rate, filling with 0 seems appropriate.
total_2015['parent_response_rate'] = total_2015['parent_response_rate'].fillna(0)
total_2015['teacher_response_rate'] = total_2015['teacher_response_rate'].fillna(0)
total_2015['student_response_rate'] = total_2015['student_response_rate'].fillna(0)

In [9]:
total_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1861 entries, 0 to 1860
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dbn                    1861 non-null   object 
 1   school_type            1861 non-null   object 
 2   school_name            1861 non-null   object 
 3   parent_response_rate   1861 non-null   float64
 4   teacher_response_rate  1861 non-null   float64
 5   student_response_rate  1861 non-null   float64
 6   collab_teachers        1859 non-null   float64
 7   leadership             1859 non-null   float64
 8   instruction            1855 non-null   float64
 9   ties                   1859 non-null   float64
 10  support                1859 non-null   float64
 11  trust                  1859 non-null   float64
dtypes: float64(9), object(3)
memory usage: 174.6+ KB


In [10]:
total_2016 = pd.read_csv('../data/survey_csv/2016_total.csv')

In [11]:
total_2016.head()

,DBN,School Name,Total Parent \nResponse Rate,Total Teacher Response Rate,Total Student Response Rate,Percent Satisfaction: Collaborative Teachers,Percent Satisfaction: Effective School Leadership,Percent Satisfaction: Rigorous Instruction,Percent Satisfaction: Strong Family-Community Ties,Percent Satisfaction: Supportive Environment,Percent Satisfaction: Trust,Percent Satisfaction: Parent,Percent Satisfaction: Student,Percent Satisfaction: Teacher
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01M015,P.S. 015 Roberto Clemente,0.36,0.96,NaN,0.96,0.94,0.88,0.92,0.78,0.94,0.92,NaN,0.90
3,01M019,P.S. 019 Asher Levy,1.00,0.90,NaN,0.91,0.91,0.94,0.92,0.92,0.94,0.93,NaN,0.92
4,01M020,P.S. 020 Anna Silver,0.40,0.59,NaN,0.62,0.33,0.58,0.80,0.73,0.55,0.72,NaN,0.56


In [12]:
# because not all surveys have the parent specific columns in the total dataframe, I am dropping those columns
total_2016 = total_2016[total_2016.columns[:-3]]

In [13]:
total_cols2 = [
    'dbn',
    'school_name',
    'parent_response_rate', 
    'teacher_response_rate', 
    'student_response_rate',
    'collab_teachers',
    'leadership',
    'instruction',
    'ties',
    'support',
    'trust'
]

In [14]:
# rename columns
total_2016.columns = total_cols2

# drop rows with all nulls
total_2016.dropna(how='all', axis=0, inplace=True)

# reset index
total_2016.reset_index(drop=True, inplace=True)

In [15]:
# I will assume that all missing data are because of no response, so percent response rate, filling with 0 seems appropriate.
total_2016['parent_response_rate'] = total_2016['parent_response_rate'].fillna(0)
total_2016['teacher_response_rate'] = total_2016['teacher_response_rate'].fillna(0)
total_2016['student_response_rate'] = total_2016['student_response_rate'].fillna(0)

In [16]:
total_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3098 entries, 0 to 3097
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dbn                    3098 non-null   object 
 1   school_name            3098 non-null   object 
 2   parent_response_rate   3098 non-null   float64
 3   teacher_response_rate  3098 non-null   float64
 4   student_response_rate  3098 non-null   float64
 5   collab_teachers        1862 non-null   float64
 6   leadership             1862 non-null   float64
 7   instruction            1862 non-null   object 
 8   ties                   1862 non-null   float64
 9   support                1862 non-null   object 
 10  trust                  1862 non-null   float64
dtypes: float64(7), object(4)
memory usage: 266.4+ KB


In [17]:
total_2017 = pd.read_csv('../data/survey_csv/2017_total.csv')

In [18]:
total_2017.head()

,DBN,School Name,Total Parent \nResponse Rate,Total Teacher Response Rate,Total Student Response Rate,Collaborative Teachers Score,Effective School Leadership Score,Rigorous Instruction Score,Supportive Environment Score,Strong Family-Community Ties Score,Trust Score
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01M015,P.S. 015 Roberto Clemente,88.0,100.0,NaN,NaN,NaN,NaN,4.25,NaN,4.33
3,01M019,P.S. 019 Asher Levy,100.0,100.0,NaN,4.45,4.55,4.81,3.53,3.62,3.73
4,01M020,P.S. 020 Anna Silver,60.0,91.0,NaN,2.50,3.01,1.81,2.30,2.97,2.68


In [19]:
total_cols3 = [
    'dbn',
    'school_name',
    'parent_response_rate', 
    'teacher_response_rate', 
    'student_response_rate',
    'collab_teachers',
    'leadership',
    'instruction',
    'support',
    'ties',
    'trust'
]

In [20]:
# rename columns
total_2017.columns = total_cols3

# drop rows with all nulls
total_2017.dropna(how='all', axis=0, inplace=True)

# reset index
total_2017.reset_index(drop=True, inplace=True)


In [21]:
# I will assume that all missing data are because of no response, so percent response rate, filling with 0 seems appropriate.
total_2017['parent_response_rate'] = total_2017['parent_response_rate'].fillna(0)
total_2017['teacher_response_rate'] = total_2017['teacher_response_rate'].fillna(0)
total_2017['student_response_rate'] = total_2017['student_response_rate'].fillna(0)

In [22]:
# check
total_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3254 entries, 0 to 3253
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dbn                    3254 non-null   object 
 1   school_name            3254 non-null   object 
 2   parent_response_rate   3254 non-null   float64
 3   teacher_response_rate  3254 non-null   float64
 4   student_response_rate  3254 non-null   float64
 5   collab_teachers        1548 non-null   float64
 6   leadership             1548 non-null   float64
 7   instruction            1548 non-null   float64
 8   support                1747 non-null   float64
 9   ties                   1531 non-null   float64
 10  trust                  1771 non-null   float64
dtypes: float64(9), object(2)
memory usage: 279.8+ KB


In [23]:
total_cols3 = [
    'dbn',
    'school_name',
    'parent_response_rate', 
    'teacher_response_rate', 
    'student_response_rate',
    'collab_teachers',
    'leadership',
    'instruction',
    'support',
    'ties',
    'trust'
]

In [24]:
total_2018 = pd.read_csv('../data/survey_csv/2018_total.csv')

In [25]:
total_2018.head()

,DBN,School Name,Total Parent \nResponse Rate,Total Teacher Response Rate,Total Student Response Rate,Collaborative Teachers Score,Effective School Leadership Score,Rigorous Instruction Score,Supportive Environment Score,Strong Family-Community Ties Score,Trust Score
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01M015,P.S. 015 ROBERTO CLEMENTE,75.0,100.0,NaN,4.12,4.09,3.91,NaN,3.80,3.99
2,01M019,P.S. 019 ASHER LEVY,96.0,95.0,NaN,4.27,4.25,4.64,NaN,4.10,3.55
3,01M020,P.S. 020 ANNA SILVER,50.0,89.0,NaN,2.76,3.04,1.94,NaN,3.64,3.12
4,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,29.0,78.0,92.0,2.40,2.45,2.04,NaN,3.33,2.32


In [26]:
# rename columns 
total_2018.columns = total_cols3

#drop rows with all nulls
total_2018.dropna(how='all', axis=0, inplace=True)

# reset index
total_2018.reset_index(drop=True, inplace=True)

In [27]:
# I will assume that all missing data are because of no response, so percent response rate, filling with 0 seems appropriate.
total_2018['parent_response_rate'] = total_2018['parent_response_rate'].fillna(0)
total_2018['teacher_response_rate'] = total_2018['teacher_response_rate'].fillna(0)
total_2018['student_response_rate'] = total_2018['student_response_rate'].fillna(0)

In [28]:
total_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1902 entries, 0 to 1901
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dbn                    1902 non-null   object 
 1   school_name            1902 non-null   object 
 2   parent_response_rate   1902 non-null   float64
 3   teacher_response_rate  1902 non-null   float64
 4   student_response_rate  1902 non-null   float64
 5   collab_teachers        1508 non-null   float64
 6   leadership             1508 non-null   float64
 7   instruction            1507 non-null   float64
 8   support                0 non-null      float64
 9   ties                   1492 non-null   float64
 10  trust                  1731 non-null   float64
dtypes: float64(9), object(2)
memory usage: 163.6+ KB


In [29]:
total_2019 = pd.read_csv('../data/survey_csv/2019_total.csv')

In [30]:
total_2019.head()

,DBN,School Name,Total Parent \nResponse Rate,Total Teacher Response Rate,Total Student Response Rate,Collaborative Teachers Score,Effective School Leadership Score,Rigorous Instruction Score,Supportive Environment Score,Strong Family-Community Ties Score,Trust Score
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01M015,P.S. 015 ROBERTO CLEMENTE,91.0,100.0,NaN,4.10,4.19,4.02,NaN,4.18,3.96
3,01M019,P.S. 019 ASHER LEVY,100.0,93.0,NaN,4.53,4.51,4.80,NaN,4.66,3.76
4,01M020,P.S. 020 ANNA SILVER,58.0,90.0,NaN,2.71,2.98,1.92,NaN,3.84,3.14


In [31]:
total_2019.columns = total_cols3

In [32]:
total_2019.dropna(how='all', axis=0, inplace=True)

In [33]:
total_2019.reset_index(drop=True, inplace=True)

In [34]:
# I will assume that all missing data are because of no response, so percent response rate, filling with 0 seems appropriate.
total_2019['parent_response_rate'] = total_2019['parent_response_rate'].fillna(0)
total_2019['teacher_response_rate'] = total_2019['teacher_response_rate'].fillna(0)
total_2019['student_response_rate'] = total_2019['student_response_rate'].fillna(0)

In [35]:
total_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1903 entries, 0 to 1902
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dbn                    1903 non-null   object 
 1   school_name            1903 non-null   object 
 2   parent_response_rate   1903 non-null   float64
 3   teacher_response_rate  1903 non-null   float64
 4   student_response_rate  1903 non-null   float64
 5   collab_teachers        1625 non-null   float64
 6   leadership             1625 non-null   float64
 7   instruction            1625 non-null   float64
 8   support                0 non-null      float64
 9   ties                   1610 non-null   float64
 10  trust                  1757 non-null   float64
dtypes: float64(9), object(2)
memory usage: 163.7+ KB


In [36]:
# reorder columns, so they are all in the same order
# first, drop the `school_type` column from 2015 dataset (it's the only one that has it)
# save and write that data to file for later use before dropping the abovementioned columns
school_types = total_2015.copy().iloc[:, :3]
school_types.to_csv('../data/school_types_2015.csv')

# drop the column
total_2015.drop(columns=['school_type'])

# reorder all years
total_2015 = total_2015.loc[:, total_cols3]
total_2016 = total_2016.loc[:, total_cols3]
total_2017 = total_2017.loc[:, total_cols3]
total_2018 = total_2018.loc[:, total_cols3]
total_2019 = total_2019.loc[:, total_cols3]

In [37]:
# write to file 
# a directory first
if not os.path.exists('../data/clean_survey'):
    os.mkdir('../data/clean_survey')

total_2015.to_csv('../data/clean_survey/2015_total.csv', index=False)
total_2016.to_csv('../data/clean_survey/2016_total.csv', index=False)
total_2017.to_csv('../data/clean_survey/2017_total.csv', index=False)
total_2018.to_csv('../data/clean_survey/2018_total.csv', index=False)
total_2019.to_csv('../data/clean_survey/2019_total.csv', index=False)

### Students' Surveys

In [38]:
student_2015 = pd.read_csv('../data/survey_csv/2015_student.csv', header=[0, 1])
student_2016 = pd.read_csv('../data/survey_csv/2016_student.csv', header=[0, 1])
student_2017 = pd.read_csv('../data/survey_csv/2017_student.csv', header=[0, 1])
student_2018 = pd.read_csv('../data/survey_csv/2018_student.csv', header=[0, 1])
student_2019 = pd.read_csv('../data/survey_csv/2019_student.csv', header=[0, 1])

In [39]:
student_2015.shape 

(1136, 302)

In [40]:
student_2015.columns.get_level_values(0).unique()

Index(['DBN', 'School Name', 'Total Student Response Rate',
       '1a. How many of the students in your class(es) feel it is important to come to school every day?',
       '1b. How many of the students in your class(es) feel it is important to pay attention in class?',
       '1c. How many of the students in your class(es) think doing homework is important?',
       '1d. How many of the students in your class(es) try hard to get good grades?',
       '2a. Think about the class that you attend first on Mondays. In that class, to what extent are you challenged?',
       '2b. Think about the class that you attend first on Mondays. In that class, to what extent do you have to work hard to do well?',
       '2c. Think about the class that you attend first on Mondays. In that class, to what extent do your teachers ask difficult questions on tests?',
       '2d. Think about the class that you attend first on Mondays. In that class, to what extent do your teachers ask difficult questions in 

In [41]:
# how many questions
len(student_2015.columns.get_level_values(0).unique())

75

In [42]:
# what are answer options
student_2015.columns.get_level_values(1)[3:59]

Index(['None', 'Some', 'A lot', 'All', 'None', 'Some', 'A lot', 'All', 'None',
       'Some', 'A lot', 'All', 'None', 'Some', 'A lot', 'All', 'Never',
       'Rarely', 'Sometimes', 'Almost always', 'Never', 'Rarely', 'Sometimes',
       'Almost always', 'Never', 'Rarely', 'Sometimes', 'Almost always',
       'Never', 'Rarely', 'Sometimes', 'Almost always', 'Never', 'Rarely',
       'Sometimes', 'Almost always', 'Never', 'Rarely', 'Sometimes',
       'Almost always', 'Never', 'Rarely', 'Sometimes', 'Almost always',
       'Never', 'Rarely', 'Sometimes', 'Almost always', 'Never', 'Rarely',
       'Sometimes', 'Almost always', 'Never', 'Rarely', 'Sometimes',
       'Almost always'],
      dtype='object')

In [43]:
student_2015.columns.get_level_values(1)[59:79]

Index(['Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree'],
      dtype='object')

In [44]:
student_2015.columns.get_level_values(1)[-10:]

Index(['Agree', 'Strongly agree', 'Strongly disagree', 'Disagree', 'Agree',
       'Strongly agree', 'Strongly disagree', 'Disagree', 'Agree',
       'Strongly agree'],
      dtype='object')

In [45]:
student_2015.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'None', 'Some', 'A lot', 'All', 'Never', 'Rarely', 'Sometimes',
       'Almost always', 'Strongly disagree', 'Disagree', 'Agree',
       'Strongly agree', 'I don't know', 'None of the time',
       'Some of the time', 'Most of the time', 'All of the time'],
      dtype='object')

In [46]:
len(student_2015.columns.get_level_values(1))

302

In [47]:
student_2015.loc[:, student_2015.columns.get_level_values(1) == "I don't know"]

,10a. How much do you agree with the following statements? My teachers incorporate students' cultures/ backgrounds into the curriculum to make learning more meaningful.,10b. How much do you agree with the following statements? I see people of many cultures/backgrounds represented in the curriculum.,10c. How much do you agree with the following statements? My teachers connect to students of different cultures/backgrounds.,10d. How much do you agree with the following statements? My teachers can communicate well with parents/guardians from different cultures/backgrounds.,10e. How much do you agree with the following statements? My teachers appreciate my culture/background.,"10f. How much do you agree with the following statements? This school offers a wide enough variety of programs, classes and activities to keep me interested in school.","10g. How much do you agree with the following statements? The programs, classes, and activities at this school encourage students to develop talent outside academics.",10h. How much do you agree with the following statements? This school is kept clean.,10i. How much do you agree with the following statements? Most of the teaching staff at this school are teaching me the organizational skills and work habits (like note-taking and keeping track of assignments) that I need to succeed in school.,10j. How much do you agree with the following statements? Most students at this school treat each other with respect.,"10k. How much do you agree with the following statements? At this school students with disabilities are included in all school activities (lunch, class trips, etc.)."
,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know
0,34.0,25.0,20.0,30.0,27.0,14.0,15.0,8.0,20.0,14.0,44.0
1,32.0,30.0,26.0,25.0,25.0,13.0,18.0,20.0,14.0,13.0,30.0
2,40.0,30.0,32.0,25.0,27.0,9.0,14.0,12.0,9.0,10.0,50.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,26.0,21.0,22.0,20.0,23.0,14.0,14.0,13.0,13.0,16.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...
1131,21.0,13.0,17.0,21.0,25.0,4.0,10.0,8.0,6.0,6.0,46.0
1132,26.0,19.0,18.0,20.0,33.0,16.0,21.0,12.0,14.0,16.0,30.0
1133,24.0,15.0,22.0,22.0,33.0,12.0,15.0,11.0,10.0,13.0,39.0


It looks like there are 4 options for every question except for the questions starting with '10'. The '10s' have 5 options. 

In [48]:
# grab the first part where it has the question number right before the .
questions = student_2015.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+\w)\.(.+)', question).group(1) for question in questions]

In [49]:
# verify that questions starting in 10 finish with a k
for q_num in q_nums:
    if q_num.startswith('10'):
        print(q_num)

10a
10b
10c
10d
10e
10f
10g
10h
10i
10j
10k


In [50]:
# looks good, so let me make new column names with appriate number attached.
# if the question has 4 answer options, the question number will be followed by numbers 1-4
# if the question has 5 answer options, the question number will be followed by numbers 1-5

# start an empty list for the new column names
new_cols = []
for q_num in q_nums:
    if q_num.startswith('10'):
        for i in range(1, 6):
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'student_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

In [51]:
len(new_cols)

302

In [52]:
student_2015.columns = new_cols

In [53]:
student_2016.head(2)

DBN                     School Name  \
  Unnamed: 0_level_1              Unnamed: 1_level_1   
0             01M034  P.S. 034 Franklin D. Roosevelt   
1             01M140          P.S. 140 Nathan Straus   

  Total Student Response Rate  \
           Unnamed: 2_level_1   
0                        0.95   
1                        0.91   

  1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.  \
                                                                                                  Strongly disagree   
0                                               0.07                                                                  
1                                               0.05                                                                  

                                              \
  Disagree Agree Strongly agree I don't know   
0     0.14  0.59           0.12         0.08   
1     0.08  0.56           0.24         0.07   

  1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.  \
                                                                                                 Strongly disagree   
0                                               0.06                                                                 
1                                               0.03                                                                 

            ...  \
  Disagree  ...   
0     0.20  ...   
1     0.15  ...   

  10d. If you are a student in grades 9-12, ANSWER this question. Adults at this school (including teachers, administrators, counselors, and the principal) help me choose which colleges to apply to.  \
                                                                                                                                                                                                 Agree   
0                                                NaN                                                                                                                                                     
1                                                NaN                                                                                                                                                     

                  \
  Strongly agree   
0            NaN   
1            NaN   

  10e. If you are a student in grades 9-12, ANSWER this question. Adults at this school (including teachers, administrators, counselors, and the principal) talk with me about how to pay for college.   \
                                                                                                                                                                                      Strongly disagree   
0                                                NaN                                                                                                                                                      
1                                                NaN                                                                                                                                                      

                                 \
  Disagree Agree Strongly agree   
0      NaN   NaN            NaN   
1      NaN   NaN            NaN   

  10f. If you are a student in grades 9-12, ANSWER this question. Adults at this school (including teachers, administrators, counselors, and the principal) help me plan for how to meet my future career goals  \
                                                                                                                                                                                              Strongly disagree   
0                                                NaN                                                                                                                         

In [54]:
student_2016.shape

(1137, 260)

In [55]:
student_2016.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'I don't know', 'None', 'A few', 'Most', 'All', 'Most ',
       'None of the time', 'Some of the time', 'Most of the time',
       'All of the time'],
      dtype='object')

In [56]:
# which questions have the 'I don't know option'
student_2016.loc[:1, student_2016.columns.get_level_values(1) == "I don't know"]

,"1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.","1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.",1c. This school is kept clean.,1d. Most students at this school treat each other with respect.,"1e. At this school students with disabilities are included in all school activities (lunch, class trips, etc.).",1f. My teachers use examples of students’ different cultures/backgrounds/families in their lessons to make learning more meaningful for me.,"1g. I see people of many races, ethnicities, cultures, and backgrounds represented in the curriculum.","1h. My teachers call on students of different races, ethnicities, cultures, and backgrounds.",1i. I feel that my teachers respect my culture/background.
,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know
0,0.08,0.15,0.08,0.13,0.43,0.15,0.17,0.12,0.10
1,0.07,0.11,0.12,0.12,0.25,0.19,0.28,0.18,0.13


In [57]:
student_2016.columns.get_level_values(1)[3:19]

Index(['Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'I don't know', 'Strongly disagree', 'Disagree', 'Agree',
       'Strongly agree', 'I don't know', 'Strongly disagree', 'Disagree',
       'Agree', 'Strongly agree', 'I don't know', 'Strongly disagree'],
      dtype='object')

In [58]:
student_2016.columns.get_level_values(0).unique()[:5]

Index(['DBN', 'School Name', 'Total Student Response Rate',
       '1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.',
       '1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.'],
      dtype='object')

In [59]:
len(student_2016.columns.get_level_values(0).unique())

65

this time the questions starting in 1 have 5 answer options

In [60]:
# grab the first part where it has the question number right before the .
questions = student_2016.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+\w)\.(.+)', question).group(1) for question in questions]

In [61]:
# if the question has 4 answer options, the question number will be followed by numbers 1-4
# if the question has 5 answer options, the question number will be followed by numbers 1-5

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if re.match('1[a-z]', q_num):
        for i in range(1, 6):
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'student_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

In [62]:
# reassign column names
student_2016.columns = new_cols

In [63]:
student_2017.head(2)

DBN                     School Name  \
  Unnamed: 0_level_1              Unnamed: 1_level_1   
0             01M034  P.S. 034 Franklin D. Roosevelt   
1             01M140          P.S. 140 Nathan Straus   

  Total Student Response Rate  \
           Unnamed: 2_level_1   
0                          96   
1                          95   

  1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.  \
                                                                                                  Strongly disagree   
0                                               11.0                                                                  
1                                                4.0                                                                  

                                              \
  Disagree Agree Strongly agree I don't know   
0     24.0  43.0           12.0          9.0   
1     16.0  49.0           22.0         10.0   

  1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.  \
                                                                                                 Strongly disagree   
0                                                6.0                                                                 
1                                                2.0                                                                 

            ...  \
  Disagree  ...   
0     23.0  ...   
1     22.0  ...   

  11a. If you are a student in grades 9-12, ANSWER this question. How much do YOU agree with the following statements? Adults at this school (including teachers, administrators, counselors, and the principal) help me consider which colleges to apply to.  \
                                                                                                                                                                                                                                            Strongly disagree   
0                                                NaN                                                                                                                                                                                                            
1                                                NaN                                                                                                                                                                                                            

                                     \
  Disagree Agree Strongly agree N/A   
0      NaN   NaN            NaN NaN   
1      NaN   NaN            NaN NaN   

  11b. If you are a student in grades 9-12, ANSWER this question. How much do YOU agree with the following statements? Adults at this school (including teachers, administrators, counselors, and the principal) show me options for how to pay for college (scholarship, grants, loans, work study programs, etc.).  \
                                                                                                                                                                                                                                                                                                   Strongly disagree   
0                                                NaN                                                                                                                                                                                                                                                                   
1                                                NaN                                                                                                                                                                                                                                                                   

    

In [64]:
student_2017.shape

(1121, 268)

In [65]:
student_2017.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'I don't know', 'None', 'A few', 'Most', 'All', 'Most ',
       'None of the time', 'Rarely', 'Some of the time', 'Most of the time',
       'N/A'],
      dtype='object')

In [66]:
student_2017.loc[:1, student_2017.columns.get_level_values(1) == "I don't know"]

,"1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.","1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.",1c. This school is kept clean.,1d. Most students at this school treat each other with respect.,"1e. At this school students with disabilities are included in all school activities (lunch, class trips, etc.).",1f. My teachers use examples of students’ different cultures/backgrounds/families in their lessons to make learning more meaningful for me.,"1g. I see people of many races, ethnicities, cultures, and backgrounds represented in the curriculum.","1h. In general, my teachers treat students from different cultures or backgrounds equally.",1i. I feel that my teachers respect my culture/background.,"1j. In general, my teachers make their lessons relevant to my everday life experiences.","1k. In general, my teachers present positive images of people from a variety of races, ethnicities, cultures, and backgrounds."
,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know
0,9.0,14.0,14.0,9.0,37.0,13.0,17.0,4.0,12.0,18.0,20.0
1,10.0,11.0,10.0,11.0,28.0,18.0,14.0,11.0,16.0,15.0,24.0


Questions starting with 1 have 'I don't know' as answer option in addition to others

In [67]:
student_2017.loc[:1, student_2017.columns.get_level_values(1) == "N/A"]

,"11a. If you are a student in grades 9-12, ANSWER this question. How much do YOU agree with the following statements? Adults at this school (including teachers, administrators, counselors, and the principal) help me consider which colleges to apply to.","11b. If you are a student in grades 9-12, ANSWER this question. How much do YOU agree with the following statements? Adults at this school (including teachers, administrators, counselors, and the principal) show me options for how to pay for college (scholarship, grants, loans, work study programs, etc.)."
,N/A,N/A
0,NaN,NaN
1,NaN,NaN


Questions starting with 11 have the NA category.

In [68]:
# grab the first part where it has the question number right before the .
questions = student_2017.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+\w)\.(.+)', question).group(1) for question in questions]

In [69]:
# if the question has 4 answer options, the question number will be followed by numbers 1-4
# if the question has 5 answer options (questions 1s), the question number will be followed by numbers 1-5
# if the question has a N/A option as an answer (Q11s), number will be assigned as 1-4 and 99 (NA).
# the 99s are assigned, so the columns can be removed later

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if re.match('1[a-z]', q_num):
        for i in range(1, 6):
            new_cols.append(q_num + '_' + str(i))
    elif q_num.startswith('11'):
        for i in [1, 2, 3, 4, 99]:
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'student_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

In [70]:
student_2017.columns = new_cols

In [71]:
# drop the columns that end with '99'
for column in student_2017.columns:
    if column.endswith('99'):
        student_2017.drop(columns=[column], inplace=True)

In [72]:
student_2018.head(2)

DBN                     School Name  \
  Unnamed: 0_level_1              Unnamed: 1_level_1   
0             01M034  P.S. 034 FRANKLIN D. ROOSEVELT   
1             01M140          P.S. 140 NATHAN STRAUS   

  Total Student Response Rate  \
           Unnamed: 2_level_1   
0                          92   
1                          79   

  1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.  \
                                                                                                  Strongly disagree   
0                                               17.0                                                                  
1                                                8.0                                                                  

                                              \
  Disagree Agree Strongly agree I don't know   
0     24.0  42.0            8.0          8.0   
1     16.0  45.0           17.0         14.0   

  1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.  \
                                                                                                 Strongly disagree   
0                                               14.0                                                                 
1                                                7.0                                                                 

            ...  \
  Disagree  ...   
0     30.0  ...   
1     14.0  ...   

  11a. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) help me consider which colleges to apply to.  \
                                                                                                                                             Strongly disagree   
0                                                NaN                                                                                                             
1                                                NaN                                                                                                             

                                     \
  Disagree Agree Strongly agree N/A   
0      NaN   NaN            NaN NaN   
1      NaN   NaN            NaN NaN   

  11b. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) show me options for how to pay for college (scholarship, grants, loans, work study programs, etc.).  \
                                                                                                                                                                                                    Strongly disagree   
0                                                NaN                                                                                                                                                                    
1                                                NaN                                                                                                                                                                    

                                     
  Disagree Agree Strongly agree N/A  
0      NaN   NaN            NaN NaN  
1      NaN   NaN            NaN NaN  

[2 rows x 268 columns]

In [73]:
student_2018.shape

(1121, 268)

In [74]:
student_2018.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'I don't know', 'None', 'A few', 'Most', 'All', 'Most ',
       'None of the time', 'Rarely', 'Some of the time', 'Most of the time',
       'N/A'],
      dtype='object')

In [75]:
student_2018.loc[:1, student_2018.columns.get_level_values(1) == "I don't know"]

,"1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.","1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.",1c. This school is kept clean.,1d. Most students at this school treat each other with respect.,"1e. At this school students with disabilities are included in all school activities (lunch, class trips, etc.).",1f. My teachers use examples of students’ different cultures/backgrounds/families in their lessons to make learning more meaningful for me.,"1g. I see people of many races, ethnicities, cultures, and backgrounds represented in the curriculum.","1h. In general, my teachers treat students from different cultures or backgrounds equally.",1i. I feel that my teachers respect my culture/background.,"1j. In general, my teachers make their lessons relevant to my everday life experiences.","1k. In general, my teachers present positive images of people from a variety of races, ethnicities, cultures, and backgrounds."
,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know
0,8.0,11.0,10.0,8.0,28.0,19.0,17.0,14.0,15.0,19.0,18.0
1,14.0,18.0,9.0,8.0,30.0,16.0,12.0,11.0,12.0,11.0,20.0


In [76]:
student_2018.loc[:1, student_2018.columns.get_level_values(1) == "N/A"]

,"11a. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) help me consider which colleges to apply to.","11b. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) show me options for how to pay for college (scholarship, grants, loans, work study programs, etc.)."
,N/A,N/A
0,NaN,NaN
1,NaN,NaN


2018 and 2017 surveys look the same, so I will follow the same manipulation for changing the column names

In [77]:
# grab the first part where it has the question number right before the .
questions = student_2018.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+\w)\.(.+)', question).group(1) for question in questions]

# if the question has 4 answer options, the question number will be followed by numbers 1-4
# if the question has 5 answer options (questions 1s), the question number will be followed by numbers 1-5
# if the question has a N/A option as an answer (Q11s), number will be assigned as 1-4 and 99 (NA).
# the 99s are assigned, so the columns can be removed later

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if re.match('1[a-z]', q_num):
        for i in range(1, 6):
            new_cols.append(q_num + '_' + str(i))
    elif q_num.startswith('11'):
        for i in [1, 2, 3, 4, 99]:
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'student_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

# change column names
student_2018.columns = new_cols

In [78]:
# drop the columns that end with '99'
for column in student_2018.columns:
    if column.endswith('99'):
        student_2018.drop(columns=[column], inplace=True)

In [79]:
student_2019.head(2)

DBN                     School Name  \
  Unnamed: 0_level_1              Unnamed: 1_level_1   
0             01M034  P.S. 034 FRANKLIN D. ROOSEVELT   
1             01M140          P.S. 140 NATHAN STRAUS   

  Total Student Response Rate  \
           Unnamed: 2_level_1   
0                        0.95   
1                        0.87   

  1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.  \
                                                                                                  Strongly disagree   
0                                               10.0                                                                  
1                                                8.0                                                                  

                                              \
  Disagree Agree Strongly agree I don't know   
0     20.0  39.0           16.0         15.0   
1     28.0  38.0           11.0         14.0   

  1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.  \
                                                                                                 Strongly disagree   
0                                                5.0                                                                 
1                                                5.0                                                                 

            ...  \
  Disagree  ...   
0     23.0  ...   
1     22.0  ...   

  11a. Adults at this school (including teachers, administrators, counselors, and the principal) help me consider which colleges to apply to.  \
                                                                                                                            Strongly disagree   
0                                                NaN                                                                                            
1                                                NaN                                                                                            

                                     \
  Disagree Agree Strongly agree N/A   
0      NaN   NaN            NaN NaN   
1      NaN   NaN            NaN NaN   

  11b. Adults at this school (including teachers, administrators, counselors, and the principal) show me options for how to pay for college (scholarship, grants, loans, work study programs, etc.).  \
                                                                                                                                                                                   Strongly disagree   
0                                                NaN                                                                                                                                                   
1                                                NaN                                                                                                                                                   

                                     
  Disagree Agree Strongly agree N/A  
0      NaN   NaN            NaN NaN  
1      NaN   NaN            NaN NaN  

[2 rows x 272 columns]

In [80]:
student_2019.shape

(1109, 272)

In [81]:
student_2019.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'I don't know', 'None', 'A few', 'Most', 'All', 'Most ',
       'None of the time', 'Rarely', 'Some of the time', 'Most of the time',
       'N/A'],
      dtype='object')

The frame above doesn't look like the correct version that I want to work with. I found the correct one, so I would have to load and clean here.

In [82]:
student_2019.loc[:1, student_2019.columns.get_level_values(1) == "I don't know"]

,"1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.","1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.",1c. This school is kept clean.,1d. Most students at this school treat each other with respect.,"1e. At this school, students with disabilities are included in all school activities (lunch, class trips, etc.).",1f. My teachers use examples of students’ different cultures/backgrounds/families in their lessons to make learning more meaningful for me.,"1g. I see people of many races, ethnicities, cultures, and backgrounds represented in the curriculum.","1h. In general, my teachers treat students from different cultures or backgrounds equally.",1i. I feel that my teachers respect my culture/background.,"1j. In general, my teachers make their lessons relevant to my everyday life experiences.","1k. In general, my teachers present positive images of people from a variety of races, ethnicities, cultures, and backgrounds."
,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know
0,15.0,18.0,12.0,13.0,28.0,24.0,17.0,12.0,16.0,19.0,24.0
1,14.0,11.0,10.0,10.0,23.0,16.0,16.0,7.0,7.0,13.0,16.0


In [83]:
student_2019.loc[:1, student_2019.columns.get_level_values(1) == "N/A"]

,"11a. Adults at this school (including teachers, administrators, counselors, and the principal) help me consider which colleges to apply to.","11b. Adults at this school (including teachers, administrators, counselors, and the principal) show me options for how to pay for college (scholarship, grants, loans, work study programs, etc.)."
,N/A,N/A
0,NaN,NaN
1,NaN,NaN


In [84]:
# how many questions
len(student_2019.columns.get_level_values(0).unique()[3:])

64

2019 survey is also similar in structure with the previous 2 years. The same clean up can be used.

In [85]:
# grab the first part where it has the question number right before the .
questions = student_2019.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+\w)\.(.+)', question).group(1) for question in questions]

# if the question has 4 answer options, the question number will be followed by numbers 1-4
# if the question has 5 answer options (questions 1s), the question number will be followed by numbers 1-5
# if the question has a N/A option as an answer (Q11s), number will be assigned as 1-4 and 99 (NA).
# the 99s are assigned, so the columns can be removed later

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if re.match('1[a-z]', q_num):
        for i in range(1, 6):
            new_cols.append(q_num + '_' + str(i))
    elif q_num.startswith('11'):
        for i in [1, 2, 3, 4, 99]:
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'student_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

# change column names
student_2019.columns = new_cols

In [86]:
student_2019.head()

,dbn,school_name,student_response_rate,1a_1,1a_2,1a_3,1a_4,1a_5,1b_1,1b_2,...,11a_1,11a_2,11a_3,11a_4,11a_99,11b_1,11b_2,11b_3,11b_4,11b_99
0,01M034,P.S. 034 FRANKLIN D. ROOSEVELT,0.95,10.0,20.0,39.0,16.0,15.0,5.0,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01M140,P.S. 140 NATHAN STRAUS,0.87,8.0,28.0,38.0,11.0,14.0,5.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01M184,P.S. 184M SHUANG WEN,1.00,9.0,22.0,46.0,14.0,10.0,9.0,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01M188,P.S. 188 THE ISLAND SCHOOL,0.66,0.0,0.0,7.0,93.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01M292,ORCHARD COLLEGIATE ACADEMY,0.82,3.0,15.0,47.0,28.0,6.0,5.0,8.0,...,2.0,7.0,32.0,54.0,5.0,5.0,6.0,29.0,54.0,5.0


In [87]:
# drop the columns that end with '99'
for column in student_2019.columns:
    if column.endswith('99'):
        student_2019.drop(columns=[column], inplace=True)

In [205]:
# fill all NaNs with 0 (i'm assuming missing response is 0)
student_2015.fillna(0, inplace=True)
student_2016.fillna(0, inplace=True)
student_2017.fillna(0, inplace=True)
student_2018.fillna(0, inplace=True)
student_2019.fillna(0, inplace=True)

In [206]:
# write to file replacing the old ones
student_2015.to_csv('../data/clean_survey/2015_student.csv', index=False)
student_2016.to_csv('../data/clean_survey/2016_student.csv', index=False)
student_2017.to_csv('../data/clean_survey/2017_student.csv', index=False)
student_2018.to_csv('../data/clean_survey/2018_student.csv', index=False)
student_2019.to_csv('../data/clean_survey/2019_student.csv', index=False)

### Parents' Surveys

In [89]:
# reading parents survey data
parent_2015 = pd.read_csv('../data/survey_csv/2015_parent.csv', header=[0, 1])
parent_2016 = pd.read_csv('../data/survey_csv/2016_parent.csv', header=[0, 1])
parent_2017 = pd.read_csv('../data/survey_csv/2017_parent.csv', header=[0, 1])
parent_2018 = pd.read_csv('../data/survey_csv/2018_parent.csv', header=[0, 1])
parent_2019 = pd.read_csv('../data/survey_csv/2019_parent.csv', header=[0, 1])

In [90]:
parent_2015.head(2)

DBN                School Name Total Parent Response Rate  \
  Unnamed: 0_level_1         Unnamed: 1_level_1         Unnamed: 2_level_1   
0             01M015  P.S. 015 Roberto Clemente                       62.0   
1             01M019        P.S. 019 Asher Levy                      100.0   

  1a. Since the beginning of the school year, how often have you had an in-person parent-teacher meeting?  \
                                                                                                    Never   
0                                                2.0                                                        
1                                                5.0                                                        

                                                                  \
   Once Once a month A few times a month Weekly Almost every day   
0  33.0         39.0                18.0    3.0              5.0   
1  38.0         27.0                25.0    3.0              2.0   

  1b. Since the beginning of the school year, how often have you volunteered time to support this school (for example, volunteered in classrooms, helped with school-wide events, etc.)?  \
                                                                                                                                                                                   Never   
0                                               59.0                                                                                                                                       
1                                               52.0                                                                                                                                       

   ...  \
   ...   
0  ...   
1  ...   

  10b. If you are a parent with a child with an Individual Education Program (IEP) ANSWER this question. This school works to achieve the goals on my child's Individual Education Program (IEP).  \
                                                                                                                                                                                Somewhat disagree   
0                                                0.0                                                                                                                                                
1                                                2.0                                                                                                                                                

                                       \
  Somewhat agree Agree Strongly agree   
0           14.0  59.0           23.0   
1           20.0  27.0           45.0   

  10c. If you are a parent with a child with an Individual Education Program (IEP) ANSWER this question. This school offers a wide enough variety of activities and services (including related services and assistive and adaptive technologies where appropriate) to help improve life outcomes for my child.      \
                                                                                                                                                                                                                                                                                                  Strongly disagree   
0                                                0.0                                                                                                                                                                                                                                                                  
1                                                4.0                                                                                                                                                                                                                                                                  

 

In [91]:
parent_2015.shape

(1861, 323)

In [92]:
parent_2015.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'Never', 'Once', 'Once a month', 'A few times a month', 'Weekly',
       'Almost every day', 'Strongly disagree', 'Disagree',
       'Somewhat disagree', 'Somewhat agree', 'Agree', 'Strongly agree',
       'I don't know', 'None', 'Some', 'About half', 'Most', 'Nearly all',
       'All', 'Very dissatisfied', 'Mostly dissatisfied',
       'Somewhat dissatisfied', 'Somewhat satisfied', 'Mostly satisfied',
       'Very satisfied'],
      dtype='object')

In [93]:
# which questions have the answer option 'Never'
parent_2015.loc[:1, parent_2015.columns.get_level_values(1) == 'Never']

,"1a. Since the beginning of the school year, how often have you had an in-person parent-teacher meeting?","1b. Since the beginning of the school year, how often have you volunteered time to support this school (for example, volunteered in classrooms, helped with school-wide events, etc.)?","1c. Since the beginning of the school year, how often have you communicated with your child's teacher about your child's performance?","1d. Since the beginning of the school year, how often have you seen your child's homework, tests or quizzes?"
,Never,Never,Never,Never
0,2.0,59.0,3.0,5.0
1,5.0,52.0,4.0,2.0


In [94]:
# verify that all 1's questions have 6 answer options
for column in parent_2015.columns:
    if re.match(r'1[a-z]', column[0]):
        print(column[1])

Never
Once
Once a month
A few times a month
Weekly
Almost every day
Never
Once
Once a month
A few times a month
Weekly
Almost every day
Never
Once
Once a month
A few times a month
Weekly
Almost every day
Never
Once
Once a month
A few times a month
Weekly
Almost every day


Questions 1s have 6 answer options:   
Never, Once, Once a month, A few times a month, Weekly, Almost every day

In [95]:
# which questions have the answer option 'I don't know'
parent_2015.loc[:1, parent_2015.columns.get_level_values(1) == "I don't know"]

,"5a. This school offers a wide enough variety of courses, extracurricular activities and services to keep my child interested in school.",5b. I feel respected by my child's principal.,5c. I trust the principal at his or her word.,5d. The principal is an effective manager who makes the school run smoothly.,5e. The principal at this school works hard to build trusting relationships with parents/guardians like me.,5f. My child's school will make me aware if there are any emotional or psychological issues affecting his/her academic performance.,5g. At this school my child is safe.,5h. This school is kept clean.,6a. The School Leadership Team (SLT) at this school meets at least once per month.,6b. The School Leadership Team (SLT) at this school makes decisions that affect the whole school.,6c. The School Leadership Team (SLT) at this school provides resources to support the SLT's priorities.,6d. The School Leadership Team (SLT) at this school assesses how the school's educational programs affect student achievement.,6e. The School Leadership Team (SLT) at this school writes a high-quality Comprehensive Education Plan (CEP) for this school.,7a. How many of the students in your child's class(es) feel it is important to come to school every day?,7b. How many of the students in your child's class(es) feel it is important to pay attention in class?,7c. How many of the students in your child's class(es) think doing homework is important?,7d. How many of the students in your child's class(es) try hard to get good grades?,8a. How satisfied are you with the following? The response I get when I contact this school.,8b. How satisfied are you with the following? The education my child has received this year.,8c. How satisfied are you with the following? The overall quality of my child's teachers this year.
,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know
0,3.0,3.0,3.0,5.0,2.0,5.0,6.0,3.0,31.0,32.0,30.0,29.0,32.0,26.0,28.0,27.0,28.0,3.0,1.0,1.0
1,3.0,3.0,1.0,2.0,1.0,3.0,3.0,3.0,35.0,35.0,33.0,33.0,34.0,22.0,20.0,19.0,22.0,1.0,0.0,0.0


In [96]:
# verify that all 5s - 8s questions have 7 answer options
for column in parent_2015.columns:
    if re.match(r'[5, 6, 7, 8][a-z]', column[0]):
        print(column)     

('5a. This school offers a wide enough variety of courses, extracurricular activities and  services to keep my child interested in school.        ', 'Strongly disagree')
('5a. This school offers a wide enough variety of courses, extracurricular activities and  services to keep my child interested in school.        ', 'Disagree')
('5a. This school offers a wide enough variety of courses, extracurricular activities and  services to keep my child interested in school.        ', 'Somewhat disagree')
('5a. This school offers a wide enough variety of courses, extracurricular activities and  services to keep my child interested in school.        ', 'Somewhat agree')
('5a. This school offers a wide enough variety of courses, extracurricular activities and  services to keep my child interested in school.        ', 'Agree')
('5a. This school offers a wide enough variety of courses, extracurricular activities and  services to keep my child interested in school.        ', 'Strongly agree')
('5a. T

Questions 5s and 6s have 7 asnwer options:   
Strongly disagree, disagree, Somewhat disagree, Somewhat agree, Agree, Strongly Agree, I don't know  
They will be assigend to (in order) 1, 2, 3, 4, 5, 6, 0   

Questions 7s have these 7 answer options:   
None, Some, About half, Most, Nearly all, All, I don't know   
They will be assigend to (in order) 1, 2, 3, 4, 5, 6, 0   

Questions 8s have these 7 answer options:  
Very dissatisfied, Mostly dissatisfied, Somewhat dissatisfied, Somewhat satisfied, Mostly satisfied, Very satisfied, I don't know   
They will be assigend to (in order) 1, 2, 3, 4, 5, 6, 0   

In [97]:
# how many questions
len(parent_2015.columns.get_level_values(0).unique()[3:])

50

The rest of the questions have 6 answer options:  
Strongly disagree, Disagree, Somewhat disagree, Somewhat disagree, Agree, Strongly agree   
They will be assigend to (in order) 1, 2, 3, 4, 5, 6

In [98]:
# grab the first part where it has the question number right before the .
questions = parent_2015.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+\w)\.(.+)', question).group(1) for question in questions]


# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if re.match('1[a-z]', q_num):
        for i in [1, 2, 3, 4, 5, 0]:
            new_cols.append(q_num + '_' + str(i))
    elif re.match('[5, 6, 7, 8][a-z]', q_num):
        for i in [1, 2, 3, 4, 5, 6, 0]:
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 7):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'parent_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

# change column names
parent_2015.columns = new_cols

In [99]:
parent_2016.head(2)

DBN                School Name Total Parent Response Rate  \
  Unnamed: 0_level_1         Unnamed: 1_level_1         Unnamed: 2_level_1   
0             01M015  P.S. 015 Roberto Clemente                       0.36   
1             01M019        P.S. 019 Asher Levy                       1.00   

  1a. School staff regularly communicate with parents/guardians about how parents can help students learn.  \
                                                                                         Strongly disagree   
0                                               0.07                                                         
1                                               0.03                                                         

                                              \
  Disagree Agree Strongly agree I don't know   
0     0.02  0.47           0.44          0.0   
1     0.06  0.49           0.42          0.0   

  1b. Parents/guardians are invited to visit classrooms to observe instruction.  \
                                                              Strongly disagree   
0                                               0.04                              
1                                               0.03                              

            ...  \
  Disagree  ...   
0     0.11  ...   
1     0.04  ...   

  9c. If you are a parent of a child in Pre-K, ANSWER this question. My child’s Pre-K teacher gives me helpful ideas about how I can support my child’s learning.  \
                                                                                                                                                            Agree   
0                                               0.00                                                                                                                
1                                               0.29                                                                                                                

                  \
  Strongly agree   
0           1.00   
1           0.65   

  9d. If you are a parent of a child in Pre-K, ANSWER this question. My child’s Pre-K teacher lets me know that I can make a difference in my child’s learning.  \
                                                                                                                                              Strongly disagree   
0                                                0.0                                                                                                              
1                                                0.0                                                                                                              

                                 \
  Disagree Agree Strongly agree   
0     0.00  0.00           1.00   
1     0.06  0.35           0.59   

  9e. If you are a parent of a child in Pre-K, ANSWER this question. My child's Pre-K program has helped me consider which elementary schools will meet my child's needs.  \
                                                                                                                                                        Strongly disagree   
0                                                0.0                                                                                                                        
1                                                0.0                                                                                                                        

                                 
  Disagree Agree Strongly agree  
0     0.00  0.00           1.00  
1     0.19  0.44           0.38  

[2 rows x 207 columns]

In [100]:
parent_2016.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'I don't know', 'Never', 'Rarely', 'Sometimes', 'Often',
       'Very dissatisfied', 'Dissatisfied', 'Satisfied', 'Very satisfied',
       'Stronger school leadership', 'More hand-on learning',
       'Stronger enrichment programs (e.g. afterschool programs, clubs, teams)',
       'Stronger arts programs', 'More challenging courses',
       'Better communication with parents', 'Higher quality teaching',
       'Smaller class size', 'Safer school environment'],
      dtype='object')

In [101]:
parent_2016.loc[:1, parent_2016.columns.get_level_values(1) == 'Stronger school leadership']

,6. Which of the following improvements would you most like your school to make (Choose ONE)?
,Stronger school leadership
0,0.03
1,0.25


In [102]:
for i in parent_2016.columns.get_level_values(0):
    if i.startswith('6'):
        print(i)

6. Which of the following improvements would you most like your school to make (Choose ONE)?
6. Which of the following improvements would you most like your school to make (Choose ONE)?
6. Which of the following improvements would you most like your school to make (Choose ONE)?
6. Which of the following improvements would you most like your school to make (Choose ONE)?
6. Which of the following improvements would you most like your school to make (Choose ONE)?
6. Which of the following improvements would you most like your school to make (Choose ONE)?
6. Which of the following improvements would you most like your school to make (Choose ONE)?
6. Which of the following improvements would you most like your school to make (Choose ONE)?
6. Which of the following improvements would you most like your school to make (Choose ONE)?


Question 6 has 9 answer options:  
'Stronger school leadership', 'More hand-on learning', 'Stronger enrichment programs (e.g. afterschool programs, clubs, teams)', 'Stronger arts programs', 'More challenging courses', 'Better communication with parents', 'Higher quality teaching', 'Smaller class size', 'Safer school environment'

In [103]:
parent_2016.loc[:1, parent_2016.columns.get_level_values(1) == "I don't know"]

,1a. School staff regularly communicate with parents/guardians about how parents can help students learn.,1b. Parents/guardians are invited to visit classrooms to observe instruction.,1c. Parents/guardians are greeted warmly when they call or visit the school.,1d. Teachers and parents/guardians think of each other as partners in educating children.,1e. Teachers work closely with families to meet students' needs.,1f. Teachers communicate regularly with parents/guardians.,1g. The principal/school leaders encourage feedback from parents/guardians and the community through regular meetings with parent and teacher leaders.,1h. Teachers understand families' problems and concerns.,"1i. My child’s teachers incorporate materials about different races, ethnicities, cultural backgrounds, and families into the curriculum to make learning more meaningful.","1j. My child sees people of many different races, ethnicities, or cultural backgrounds represented in the curriculum.",...,1n. This school is kept clean.,1o. School Safety Agents promote a safe and respectful environment at this school.,3a. The principal/school leader at this school is strongly committed to shared decision making.,3b. The principal/school leader at this school works to create a sense of community in the school.,3c. The principal/school leader at this school promotes family and community involvement in the school.,5a. How satisfied are you with the following? The response I get when I contact this school.,5b. How satisfied are you with the following? The education my child has received this year.,5c. How satisfied are you with the following? The overall quality of my child's teachers this year.,"5d. How satisfied are you with the following? The performance of the citywide Panel for Education Policy with regard to school resources, oversight, curriculum, and progress in student achievement.","5e. How satisfied are you with the following? The performance of the Schools Chancellor with regard to school resources, oversight, curriculum, and progress in student achievement."
,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,...,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know
0,0.0,0.04,0.0,0.0,0.00,0.00,0.02,0.02,0.09,0.04,...,0.02,0.00,0.04,0.02,0.02,0.00,0.02,0.0,NaN,NaN
1,0.0,0.03,0.0,0.0,0.01,0.01,0.03,0.03,0.03,0.03,...,0.00,0.01,0.08,0.03,0.03,0.01,0.00,0.0,NaN,NaN


In [104]:
# verify that all 1s, 3s, and 5s questions have 5 answer options
for column in parent_2016.columns:
    if re.match(r'[1, 3, 5][a-z]', column[0]):
        print(column)     

('1a. School staff regularly communicate with parents/guardians about how parents can help students learn.', 'Strongly disagree')
('1a. School staff regularly communicate with parents/guardians about how parents can help students learn.', 'Disagree')
('1a. School staff regularly communicate with parents/guardians about how parents can help students learn.', 'Agree')
('1a. School staff regularly communicate with parents/guardians about how parents can help students learn.', 'Strongly agree')
('1a. School staff regularly communicate with parents/guardians about how parents can help students learn.', "I don't know")
('1b. Parents/guardians are invited to visit classrooms to observe instruction.', 'Strongly disagree')
('1b. Parents/guardians are invited to visit classrooms to observe instruction.', 'Disagree')
('1b. Parents/guardians are invited to visit classrooms to observe instruction.', 'Agree')
('1b. Parents/guardians are invited to visit classrooms to observe instruction.', 'Strongly

Questions 1s and 3s have the following 5 answer options:  
Strongly disagree, Disagree, Agree, Strongly agree, I don't know   
Questions 5s have the following 5 answer options:  
Very dissatisfied, dissatisfied, Satisfied, Very satisfied, I don't know  

In [105]:
answers = []
for column in parent_2016.columns:
#     if re.match(r'[2, 4, 7, 8, 9][a-z]', column[0]):
#    if re.match(r'[4][a-z]', column[0]):
    if re.match(r'[2, 7, 8, 9][a-z]', column[0]):
        answers.append(column[1]) 
        
print(set(answers))

{'Disagree', 'Strongly disagree', 'Strongly agree', 'Agree'}


Questions 2, 7, 8, and 9 have 4 answer options:   
strongly disagree, disagree, agree, strongly agree   
Questions 4s have these 4 answer options:   
Never, Rarely, Sometimes, Often

In [106]:
# grab the first part where it has the question number right before the .
questions = parent_2016.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+[a-z]*)\.(.+)', question).group(1) for question in questions]

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if q_num.startswith('6'):
        for i in range(1, 10):
            new_cols.append(q_num + '_' + str(i))
    elif re.match('[1, 3, 5][a-z]', q_num):
        for i in [1, 2, 3, 4, 0]:
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'parent_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

# change column names
parent_2016.columns = new_cols

In [107]:
parent_2017.head(2)

DBN                School Name Total Parent Response Rate  \
  Unnamed: 0_level_1         Unnamed: 1_level_1         Unnamed: 2_level_1   
0             01M015  P.S. 015 Roberto Clemente                         88   
1             01M019        P.S. 019 Asher Levy                        100   

  1a. School staff regularly communicate with me about how I can help my child learn.  \
                                                                    Strongly disagree   
0                                                4.0                                    
1                                                2.0                                    

                                 \
  Disagree Agree Strongly agree   
0      1.0  50.0           46.0   
1      3.0  49.0           47.0   

  1b. I am invited to visit classrooms to observe instruction.                 \
                                             Strongly disagree Disagree Agree   
0                                                5.0               14.0  54.0   
1                                                3.0                5.0  44.0   

   ...  \
   ...   
0  ...   
1  ...   

  10c. If you are a parent/guardian of a child in pre-K, ANSWER this question: My child’s Pre-K teacher lets me know that I can make a difference in my child’s learning.  \
                                                                                                                                                                    Agree   
0                                               25.0                                                                                                                        
1                                               31.0                                                                                                                        

                  \
  Strongly agree   
0           63.0   
1           50.0   

  10d. If you are a parent/guardian of a child in pre-K, ANSWER this question: My child's Pre-K teacher gives me opportunities to share what I know about my child.  \
                                                                                                                                                  Strongly disagree   
0                                                0.0                                                                                                                  
1                                                6.0                                                                                                                  

                                 \
  Disagree Agree Strongly agree   
0      0.0  63.0           38.0   
1     13.0  31.0           50.0   

  10e. If you are a parent/guardian of a child in pre-K, ANSWER this question: My child's Pre-K program has helped me consider which elementary schools will meet my child's needs.  \
                                                                                                                                                                  Strongly disagree   
0                                                0.0                                                                                                                                  
1                                                6.0                                                                                                                                  

                                 
  Disagree Agree Strongly agree  
0     13.0  50.0           38.0  
1     19.0  25.0           50.0  

[2 rows x 195 columns]

In [108]:
parent_2017.shape

(2965, 195)

In [109]:
parent_2017.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'I don't know', 'Never', 'Rarely', 'Sometimes', 'Often',
       'Very dissatisfied', 'Dissatisfied', 'Satisfied', 'Very satisfied',
       'Very unlikely', 'Somewhat unlikely', 'Somewhat likely', 'Very likely',
       'Stronger school leadership', 'More hand-on learning',
       'Stronger enrichment programs (e.g. afterschool programs, clubs, teams)',
       'Stronger arts programs', 'More challenging courses',
       'Better communication with parents/guardians',
       'Higher quality teaching', 'Smaller class size',
       'Safer school environment'],
      dtype='object')

In [110]:
parent_2017.loc[:1, parent_2017.columns.get_level_values(1) == 'Stronger school leadership']

,7. Which of the following improvements would you most like your school to make (Choose ONE)?
,Stronger school leadership
0,3.0
1,2.0


In [111]:
for i in parent_2017.columns.get_level_values(0):
    if i.startswith('7'):
        print(i)

7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?


Question 7 has 9 answer options:  
'Stronger school leadership', 'More hand-on learning', 'Stronger enrichment programs (e.g. afterschool programs, clubs, teams)', 'Stronger arts programs', 'More challenging courses', 'Better communication with parents/guardians', 'Higher quality teaching', 'Smaller class size', 'Safer school environment'

In [112]:
parent_2017.loc[:1, parent_2017.columns.get_level_values(1) == "I don't know"]

,2a. Teachers and parents/guardians think of each other as partners in educating children.,2b. The principal/school leader encourages feedback from parents/guardians and the community through regular meetings with parent/guardian and teacher leaders.,"2c. This school offers a wide enough variety of courses, extracurricular activities, and services to keep my child interested in school.",2d. My child's school will make me aware if there are any emotional or psychological issues affecting my child's academic performance.,2e. At this school my child is safe.,2f. This school is kept clean.,2g. School Safety Agents promote a safe and respectful environment at this school.,3a. The principal/school leader at this school is strongly committed to shared decision making.,3b. The principal/school leader at this school works to create a sense of community in the school.,3c. The principal/school leader at this school promotes family and community involvement in the school.,5a. The response I get when I contact this school.,5b. The education my child has received this year.,5c. The overall quality of my child's teachers this year.,"5d. The performance of the citywide Panel for Education Policy with regard to school resources, oversight, curriculum, and progress in student achievement.","5e. The performance of the Schools Chancellor with regard to school resources, oversight, curriculum, and progress in student achievement."
,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know
0,1.0,5.0,3.0,2.0,2.0,2.0,1.0,7.0,2.0,4.0,1.0,0.0,0.0,NaN,NaN
1,5.0,7.0,6.0,6.0,3.0,3.0,4.0,11.0,5.0,4.0,0.0,0.0,2.0,NaN,NaN


In [113]:
# verify that all 2s, 3s, and 5s questions have 5 answer options
for column in parent_2017.columns:
    if re.match(r'[2, 3, 5][a-z]', column[0]):
        print(column)     

('2a. Teachers and parents/guardians think of each other as partners in educating children.', 'Strongly disagree')
('2a. Teachers and parents/guardians think of each other as partners in educating children.', 'Disagree')
('2a. Teachers and parents/guardians think of each other as partners in educating children.', 'Agree')
('2a. Teachers and parents/guardians think of each other as partners in educating children.', 'Strongly agree')
('2a. Teachers and parents/guardians think of each other as partners in educating children.', "I don't know")
('2b. The principal/school leader encourages feedback from parents/guardians and the community through regular meetings with parent/guardian and teacher leaders. ', 'Strongly disagree')
('2b. The principal/school leader encourages feedback from parents/guardians and the community through regular meetings with parent/guardian and teacher leaders. ', 'Disagree')
('2b. The principal/school leader encourages feedback from parents/guardians and the commun

Questions 2s and 3s have the following 5 options:   
Strongly disagree, disagree, agree, strongly agree, I don't know  
Questions 5s have the following 5 answer options:   
Very dissatisfied, dissatisfied, satisfied, very satisfied, I don't know

In [114]:
answers = []
for column in parent_2017.columns:
#     if re.match(r'[1, 4, 6, 8, 9, 10][a-z]', column[0]):
    if re.match(r'[1][a-z]', column[0]):
#    if re.match(r'[4][a-z]', column[0]):

        answers.append(column[1]) 
        
print(set(answers))

{'Disagree', 'Strongly disagree', 'Strongly agree', 'Agree'}


Questions 4s have the following 4 options:  
Never, Rarely, Sometimes, Often 
Questions 6s have the following 4 options:  
Very unlikely, Somewhat unlikely, Somewhat likely, Very likely.   
Questions 1s, 8s, 9s, and 10s have the following 4 options:   
Strongly disagree, Disagree, Agree, Strongly agree   

In [115]:
# grab the first part where it has the question number right before the .
questions = parent_2017.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+[a-z]*)\.(.+)', question).group(1) for question in questions]

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if q_num.startswith('7'):
        for i in range(1, 10):
            new_cols.append(q_num + '_' + str(i))
    elif re.match('[2, 3, 5][a-z]', q_num):
        for i in [1, 2, 3, 4, 0]:
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'parent_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

# change column names
parent_2017.columns = new_cols

In [116]:
parent_2018.head(2)

DBN                School Name Total Parent Response Rate  \
  Unnamed: 0_level_1         Unnamed: 1_level_1         Unnamed: 2_level_1   
0             01M015  P.S. 015 ROBERTO CLEMENTE                         75   
1             01M019        P.S. 019 ASHER LEVY                         96   

  1a. School staff regularly communicate with me about how I can help my child learn.  \
                                                                    Strongly disagree   
0                                                5.0                                    
1                                                2.0                                    

                                 \
  Disagree Agree Strongly agree   
0      2.0  45.0           49.0   
1      3.0  50.0           44.0   

  1b.  My child's school offers me opportunities to visit my child's classroom, such as observing instruction, participating in an activity with my child, etc.  \
                                                                                                                                              Strongly disagree   
0                                                6.0                                                                                                              
1                                                2.0                                                                                                              

                  ...  \
  Disagree Agree  ...   
0      5.0  42.0  ...   
1      6.0  44.0  ...   

  10c. For Pre-K and 3-K: My child’s teacher lets me know that I can make a difference in my child’s learning.  \
                                                                                                         Agree   
0                                               44.0                                                             
1                                               25.0                                                             

                  \
  Strongly agree   
0           56.0   
1           75.0   

  10d. For Pre-K and 3-K: My child’s teacher gives me opportunities to share what I know about my child.  \
                                                                                       Strongly disagree   
0                                                0.0                                                       
1                                                0.0                                                       

                                 \
  Disagree Agree Strongly agree   
0      0.0  25.0           75.0   
1      0.0  25.0           75.0   

  10e. For Pre-K and 3-K: Someone at my child’s pre-K or 3-K program has helped me consider which schools or programs would be best for my child for next school year.  \
                                                                                                                                                     Strongly disagree   
0                                               11.0                                                                                                                     
1                                                0.0                                                                                                                     

                                 
  Disagree Agree Strongly agree  
0     11.0  22.0           56.0  
1     13.0  25.0           63.0  

[2 rows x 195 columns]

In [117]:
parent_2018.shape

(2889, 195)

In [118]:
parent_2018.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'I don't know', 'Never', 'Rarely', 'Sometimes', 'Often',
       'Very dissatisfied', 'Dissatisfied', 'Satisfied', 'Very satisfied',
       'Very unlikely', 'Somewhat unlikely', 'Somewhat likely', 'Very likely',
       'Stronger school leadership', 'More hand-on learning',
       'Stronger enrichment programs (e.g. afterschool programs, clubs, teams)',
       'Stronger arts programs', 'More challenging courses',
       'Better communication with parents/guardians',
       'Higher quality teaching', 'Smaller class size',
       'Safer school environment'],
      dtype='object')

In [119]:
parent_2018.loc[:1, parent_2018.columns.get_level_values(1) == "I don't know"]

,2a. Teachers and parents/guardians think of each other as partners in educating children.,2b. The principal/school leader encourages feedback from parents/guardians and the community through regular meetings with parent/guardian and teacher leaders.,"2c. This school offers a wide enough variety of courses, extracurricular activities, and services to keep my child interested in school.",2d. My child's school will make me aware if there are any emotional or psychological issues affecting my child's academic performance.,2e. At this school my child is safe.,2f. This school is kept clean.,2g. School Safety Agents promote a safe and respectful environment at this school.,3a. The principal/school leader at this school is strongly committed to shared decision making.,3b. The principal/school leader at this school works to create a sense of community in the school.,3c. The principal/school leader at this school promotes family and community involvement in the school.,5a. The response I get when I contact this school.,5b. The education my child has received this year.,5c. The overall quality of my child's teachers this year.,"5d. The performance of the citywide Panel for Education Policy with regard to school resources, oversight, curriculum, and progress in student achievement.","5e. The performance of the Schools Chancellor with regard to school resources, oversight, curriculum, and progress in student achievement."
,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know
0,2.0,0.0,1.0,4.0,2.0,1.0,2.0,9.0,2.0,2.0,1.0,3.0,4.0,NaN,NaN
1,3.0,6.0,6.0,4.0,6.0,5.0,6.0,9.0,4.0,3.0,0.0,0.0,1.0,NaN,NaN


In [120]:
# verify that all 2s, 3s, and 5s questions have 5 answer options
for column in parent_2018.columns:
    if re.match(r'[2, 3, 5][a-z]', column[0]):
        print(column)     

('2a. Teachers and parents/guardians think of each other as partners in educating children.', 'Strongly disagree')
('2a. Teachers and parents/guardians think of each other as partners in educating children.', 'Disagree')
('2a. Teachers and parents/guardians think of each other as partners in educating children.', 'Agree')
('2a. Teachers and parents/guardians think of each other as partners in educating children.', 'Strongly agree')
('2a. Teachers and parents/guardians think of each other as partners in educating children.', "I don't know")
('2b. The principal/school leader encourages feedback from parents/guardians and the community through regular meetings with parent/guardian and teacher leaders. ', 'Strongly disagree')
('2b. The principal/school leader encourages feedback from parents/guardians and the community through regular meetings with parent/guardian and teacher leaders. ', 'Disagree')
('2b. The principal/school leader encourages feedback from parents/guardians and the commun

In [121]:
for i in parent_2018.columns.get_level_values(0):
    if i.startswith('7'):
        print(i)

7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?


In [122]:
# 2017 and 2018 look like they have the same questions, so I will do the same as above to clean the column names
# grab the first part where it has the question number right before the .
questions = parent_2018.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+[a-z]*)\.(.+)', question).group(1) for question in questions]

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if q_num.startswith('7'):
        for i in range(1, 10):
            new_cols.append(q_num + '_' + str(i))
    elif re.match('[2, 3, 5][a-z]', q_num):
        for i in [1, 2, 3, 4, 0]:
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'parent_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

# change column names
parent_2018.columns = new_cols

In [123]:
parent_2019.head(2)

DBN                School Name Total Parent Response Rate  \
  Unnamed: 0_level_1         Unnamed: 1_level_1         Unnamed: 2_level_1   
0             01M015  P.S. 015 ROBERTO CLEMENTE                         91   
1             01M019        P.S. 019 ASHER LEVY                        100   

  1a. School staff regularly communicate with me about how I can help my child learn.  \
                                                                    Strongly disagree   
0                                                2.0                                    
1                                                1.0                                    

                                 \
  Disagree Agree Strongly agree   
0      3.0  42.0           53.0   
1      0.0  52.0           46.0   

  1b.  My child's school offers me opportunities to visit my child's classroom, such as observing instruction, participating in an activity with my child, etc.  \
                                                                                                                                              Strongly disagree   
0                                                2.0                                                                                                              
1                                                2.0                                                                                                              

                  ...  \
  Disagree Agree  ...   
0      7.0  41.0  ...   
1      3.0  41.0  ...   

  10c. My child’s teacher lets me know that I can make a difference in my child’s learning.  \
                                                                                      Agree   
0                                               29.0                                          
1                                               13.0                                          

                  \
  Strongly agree   
0           57.0   
1           88.0   

  10d. My child’s teacher gives me opportunities to share what I know about my child.  \
                                                                    Strongly disagree   
0                                                0.0                                    
1                                                0.0                                    

                                 \
  Disagree Agree Strongly agree   
0      0.0  57.0           43.0   
1      0.0  13.0           88.0   

  10e. Someone at my child’s pre-K or 3-K program has helped me consider which schools or programs would be best for my child for next school year.  \
                                                                                                                                  Strongly disagree   
0                                                0.0                                                                                                  
1                                                0.0                                                                                                  

                                 
  Disagree Agree Strongly agree  
0     29.0  14.0           57.0  
1     13.0  38.0           50.0  

[2 rows x 196 columns]

In [124]:
parent_2019.shape

(2946, 196)

In [125]:
parent_2019.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'Strongly disagree', 'Disagree', 'Agree', 'Strongly agree',
       'I don't know', 'Never', 'Rarely', 'Sometimes', 'Often',
       'Very dissatisfied', 'Dissatisfied', 'Satisfied', 'Very satisfied',
       'Very unlikely', 'Somewhat unlikely', 'Somewhat likely', 'Very likely',
       'Stronger school leadership', 'More hand-on learning',
       'Stronger enrichment programs (e.g. afterschool programs, clubs, teams)',
       'Stronger arts programs', 'More challenging courses',
       'Better communication with parents/guardians',
       'Higher quality teaching', 'Smaller class size',
       'Safer school environment'],
      dtype='object')

In [126]:
parent_2019.loc[:1, parent_2019.columns.get_level_values(1) == "I don't know"]

,2a. Teachers and parents/guardians think of each other as partners in educating children.,2b. The principal/school leader at this school works hard to build trusting relationships with parents/ guardians like me.,2c. The principal/school leader encourages feedback from parents/guardians and the community through regular meetings with parent/guardian and teacher leaders.,"2d. This school offers a wide enough variety of programs, classes, and activities to keep my child interested in school.",2e. My child's school will make me aware if there are any emotional or psychological issues affecting my child's academic performance.,2f. At this school my child is safe.,2g. The school is kept clean.,2h. School Safety Agents promote a safe and respectful environment at this school.,3a. The principal/school leader at this school is strongly committed to shared decision making.,3b. The principal/school leader at this school works to create a sense of community in the school.,3c. The principal/school leader at this school promotes family and community involvement in the school.,5a. The response I get when I contact this school.,5b. The education my child has received this year.,5c. The overall quality of my child's teachers this year.,"5d. The performance of the citywide Panel for Education Policy with regard to school resources, oversight, curriculum, and progress in student achievement.","5e. The performance of the Schools Chancellor with regard to school resources, oversight, curriculum, and progress in student achievement."
,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know
0,5.0,2.0,3.0,7.0,5.0,2.0,3.0,5.0,6.0,2.0,2.0,0.0,1.0,1.0,NaN,NaN
1,3.0,4.0,4.0,5.0,3.0,2.0,2.0,2.0,8.0,7.0,5.0,0.0,1.0,1.0,NaN,NaN


In [127]:
# verify that all 2s, 3s, and 5s questions have 5 answer options
for column in parent_2019.columns:
    if re.match(r'[2, 3, 5][a-z]', column[0]):
        print(column)     

('2a. Teachers and parents/guardians think of each other as partners in educating children.', 'Strongly disagree')
('2a. Teachers and parents/guardians think of each other as partners in educating children.', 'Disagree')
('2a. Teachers and parents/guardians think of each other as partners in educating children.', 'Agree')
('2a. Teachers and parents/guardians think of each other as partners in educating children.', 'Strongly agree')
('2a. Teachers and parents/guardians think of each other as partners in educating children.', "I don't know")
('2b. The principal/school leader at this school works hard to build trusting relationships with parents/ guardians like me.', 'Strongly disagree')
('2b. The principal/school leader at this school works hard to build trusting relationships with parents/ guardians like me.', 'Disagree')
('2b. The principal/school leader at this school works hard to build trusting relationships with parents/ guardians like me.', 'Agree')
('2b. The principal/school lead

Questions 2s and 3s have the following 5 options:  
Strongly disagree, disagree, agree, strongly agree, I don't know  
Questions 5s have the following 5 answer options:  
Very dissatisfied, dissatisfied, satisfied, very satisfied, I don't know  

In [128]:
for i in parent_2019.columns.get_level_values(0):
    if i.startswith('7'):
        print(i)

7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?
7. Which of the following improvements would you most like your school to make (Choose ONE)?


Question 7 has the following 9 options as answers:  
'Stronger school leadership', 'More hand-on learning', 'Stronger enrichment programs (e.g. afterschool programs, clubs, teams)', 'Stronger arts programs', 'More challenging courses', 'Better communication with parents/guardians', 'Higher quality teaching', 'Smaller class size', 'Safer school environment'

In [129]:
for i in parent_2019.columns.get_level_values(0):
    if i.startswith('1'):
        print(i)

1a. School staff regularly communicate with me about how I can help my child learn.
1a. School staff regularly communicate with me about how I can help my child learn.
1a. School staff regularly communicate with me about how I can help my child learn.
1a. School staff regularly communicate with me about how I can help my child learn.
1b.  My child's school offers me opportunities to visit my child's classroom, such as observing instruction, participating in an activity with my child, etc.
1b.  My child's school offers me opportunities to visit my child's classroom, such as observing instruction, participating in an activity with my child, etc.
1b.  My child's school offers me opportunities to visit my child's classroom, such as observing instruction, participating in an activity with my child, etc.
1b.  My child's school offers me opportunities to visit my child's classroom, such as observing instruction, participating in an activity with my child, etc.
1c. My child’s school offers me 

In [130]:
answers = []
for column in parent_2019.columns:
#     if re.match(r'[1, 4, 6, 8, 9, 10][a-z]', column[0]):
    if re.match(r'[6][a-z]', column[0]):
#    if re.match(r'[4][a-z]', column[0]):

        answers.append(column[1]) 
        
print(set(answers))

{'Somewhat unlikely', 'Very likely', 'Very unlikely', 'Somewhat likely'}


Questions 4s have the following 4 options:  
Never, Rarely, Sometimes, Often  
Questions 6s have the following 4 options:  
Very unlikely, Somewhat unlikely, Somewhat likely, Very likely.  
Questions 1s, 8s, 9s, and 10s have the following 4 options:  
Strongly disagree, Disagree, Agree, Strongly agree

In [131]:
# this also looks similar to 2018, except there is an extra column (not sure where, but it should work)
# grab the first part where it has the question number right before the .
questions = parent_2019.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+[a-z]*)\.(.+)', question).group(1) for question in questions]

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if q_num.startswith('7'):
        for i in range(1, 10):
            new_cols.append(q_num + '_' + str(i))
    elif re.match('[2, 3, 5][a-z]', q_num):
        for i in [1, 2, 3, 4, 0]:
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'parent_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

# change column names
parent_2019.columns = new_cols

In [207]:
# fill missing data with 0
parent_2015.fillna(0, inplace=True)
parent_2016.fillna(0, inplace=True)
parent_2017.fillna(0, inplace=True)
parent_2018.fillna(0, inplace=True)
parent_2019.fillna(0, inplace=True)

In [208]:
# write to file
parent_2015.to_csv('../data/clean_survey/2015_parent.csv', index=False)
parent_2016.to_csv('../data/clean_survey/2016_parent.csv', index=False)
parent_2017.to_csv('../data/clean_survey/2017_parent.csv', index=False)
parent_2018.to_csv('../data/clean_survey/2018_parent.csv', index=False)
parent_2019.to_csv('../data/clean_survey/2019_parent.csv', index=False)

### Teachers' Surveys

In [133]:
teacher_2015 = pd.read_csv('../data/survey_csv/2015_teacher.csv', header=[0, 1])
teacher_2016 = pd.read_csv('../data/survey_csv/2016_teacher.csv', header=[0, 1])
teacher_2017 = pd.read_csv('../data/survey_csv/2017_teacher.csv', header=[0, 1])
teacher_2018 = pd.read_csv('../data/survey_csv/2018_teacher.csv', header=[0, 1])
teacher_2019 = pd.read_csv('../data/survey_csv/2019_teacher.csv', header=[0, 1])

In [134]:
teacher_2015.head(2)

DBN                School Name Total Teacher Response Rate  \
  Unnamed: 0_level_1         Unnamed: 1_level_1          Unnamed: 2_level_1   
0             01M015  P.S. 015 Roberto Clemente                       100.0   
1             01M019        P.S. 019 Asher Levy                        98.0   

  1a. How much influence do teachers have over school policy in each of the areas below? Hiring new professional personnel.  \
                                                                                                               No influence   
0                                                8.0                                                                          
1                                               62.0                                                                          

                                                                         \
  Very little Little  Some A moderate amount A great deal of influence    
0         NaN    NaN  92.0               NaN                        NaN   
1         NaN    NaN  38.0               NaN                        NaN   

  1b. How much influence do teachers have over school policy in each of the areas below? Planning how discretionary school funds should be used.   \
                                                                                                                                     No influence   
0                                               39.0                                                                                                
1                                               49.0                                                                                                

   ...  \
   ...   
0  ...   
1  ...   

  25c. The following question is only for respondents who selected "Elementary (PreK-5)" in question 18. How many of the students in your class(es) think doing homework is important?  \
                                                                                                                                                                            About half   
0                                                NaN                                                                                                                                     
1                                                NaN                                                                                                                                     

                           \
    Most Nearly all  All    
0   68.0        NaN   NaN   
1  100.0        NaN   NaN   

  25d. The following question is only for respondents who selected "Elementary (PreK-5)" in question 18. How many of the students in your class(es) try hard to get good grades?  \
                                                                                                                                                                            None   
0                                               18.0                                                                                                                               
1                                                0.0                                                                                                                               

                                           
  Some About half   Most Nearly all  All   
0  NaN        NaN   82.0        NaN   NaN  
1  NaN        NaN  100.0        NaN   NaN  

[2 rows x 939 columns]

In [135]:
teacher_2015.shape

(1861, 939)

In [136]:
teacher_2015.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'No influence', 'Very little', 'Little', 'Some', 'A moderate amount',
       'A great deal of influence ', 'A great deal of influence', 'None',
       'A few', 'Most', 'Nearly all', 'All', 'Never', 'Once', 'Once a month',
       'Weekly', 'A few times a week', 'Almost every day', 'Rarely',
       'Sometimes ', 'Often', 'Almost always ', 'Always ', 'Strongly disagree',
       'Disagree', 'Somewhat disagree', 'Somewhat agree', 'Agree',
       'Strongly Agree', 'I don't know ', 'Strongly agree',
       'Elementary (PreK-5)', 'Middle (6-8)', 'High (9-12)', 'Sometimes',
       'Almost always', 'About half', ' All '],
      dtype='object')

In [137]:
teacher_2015.loc[:1, teacher_2015.columns.get_level_values(1) == 'High (9-12)']

,18. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)
,High (9-12)
0,NaN
1,NaN


In [138]:
for i in teacher_2015.columns.get_level_values(0):
    if i.startswith('18'):
        print(i)

18. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)
18. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)
18. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)


Question 18 has the following 3 options:
'Elementary (PreK-5)', 'Middle (6-8)', 'High (9-12)'

In [139]:
teacher_2015.loc[:1, teacher_2015.columns.get_level_values(1) == 'A great deal of influence ']

,1a. How much influence do teachers have over school policy in each of the areas below? Hiring new professional personnel.,1b. How much influence do teachers have over school policy in each of the areas below? Planning how discretionary school funds should be used.,1c. How much influence do teachers have over school policy in each of the areas below? Selecting instructional materials used in classrooms.,1d. How much influence do teachers have over school policy in each of the areas below? Developing instructional materials.
,A great deal of influence,A great deal of influence,A great deal of influence,A great deal of influence
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN


In [140]:
for column in teacher_2015.columns:
    if re.match(r'[1][a-z]', column[0]):
        print(column)

('1a. How much influence do teachers have over school policy in each of the areas below? Hiring new professional personnel.', 'No influence')
('1a. How much influence do teachers have over school policy in each of the areas below? Hiring new professional personnel.', 'Very little')
('1a. How much influence do teachers have over school policy in each of the areas below? Hiring new professional personnel.', 'Little')
('1a. How much influence do teachers have over school policy in each of the areas below? Hiring new professional personnel.', 'Some')
('1a. How much influence do teachers have over school policy in each of the areas below? Hiring new professional personnel.', 'A moderate amount')
('1a. How much influence do teachers have over school policy in each of the areas below? Hiring new professional personnel.', 'A great deal of influence ')
('1b. How much influence do teachers have over school policy in each of the areas below? Planning how discretionary school funds should be used.

Questions 1s have the following 6 options:   
'No influence', 'Very little', 'Little', 'Some', 'A moderate amount', 'A great deal of influence'

In [141]:
teacher_2015.loc[:1, teacher_2015.columns.get_level_values(1) == 'A few']

,2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?,2b. How many teachers at this school (not just in your practice) are really trying to improve their teaching?,2c. How many teachers at this school (not just in your practice) take responsibility for improving the school?,2d. How many teachers at this school (not just in your practice) feel responsible for helping students develop self-management?,2e. How many teachers at this school (not just in your practice) are willing to take risks to make the school better?,2f. How many teachers at this school (not just in your practice) are eager to try new ideas?,2g. How many teachers at this school (not just in your practice) feel responsible that all student learn?
,A few,A few,A few,A few,A few,A few,A few
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
for column in teacher_2015.columns:
    if re.match(r'[2][a-z]', column[0]):
        print(column)

('2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?', 'None')
('2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?', 'A few')
('2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?', 'Some')
('2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?', 'Most')
('2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?', 'Nearly all')
('2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?', 'All')
('2b. How many teachers at this school (not just in your practice) are really trying to improve their teaching?', 'None')
('2b. How many teachers at this school (not just in your practice) are really trying to improve their teaching?', 'A few')
('2b.

Questions 2s have the following 7 options:   
'None', 'A few', 'Little', 'Some', 'Most', 'Nearly all', 'All'

In [143]:
teacher_2015.loc[:1, teacher_2015.columns.get_level_values(1) == 'Weekly']

,"4a. For general/self-contained/literacy/science/social studies: During your last instructional unit, how often were you able to build students’ knowledge through content-rich non-fiction.","4b. For general/self-contained/literacy/science/social studies: During your last instructional unit, how often were you able to provide students reading and writing experiences grounded in evidence from text, both literary and informational.","4c. For general/self-contained/literacy/science/social studies: During your last instructional unit, how often were you able to provide students with opportunities to interact with complex/grade-level text.","4d. For general/self-contained/literacy/science/social studies: During your last instructional unit, how often were you able to provide students with opportunities to interact with academic language.","5a. For general/self-contained/math/science: During your last instructional unit, how often were you able to focus deeply on the concepts emphasized in the standards.","5b. For general/self-contained/math/science: During your last instructional unit, how often were you able to create coherent progressions within the standards from previous grades to current grade so student knowledge/skills build on previous learning.","5c. For general/self-contained/math/science: During your last instructional unit, how often were you able to create coherent progressions within the standards from current grade to next grades to prepare students for math concepts taught in later years.","5d. For general/self-contained/math/science: During your last instructional unit, how often were you able to develop students’ conceptual understanding, procedural fluency, and their ability to apply math in context."
,Weekly,Weekly,Weekly,Weekly,Weekly,Weekly,Weekly,Weekly
0,91.0,95.0,95.0,100.0,100.0,95.0,90.0,100.0
1,95.0,100.0,100.0,100.0,100.0,94.0,94.0,100.0


In [144]:
for column in teacher_2015.columns:
    if re.match(r'[4, 5][a-z]', column[0]):
        print(column)

('4a. For general/self-contained/literacy/science/social studies: During your last instructional unit, how often were you able to build students’ knowledge through content-rich non-fiction.', 'Never')
('4a. For general/self-contained/literacy/science/social studies: During your last instructional unit, how often were you able to build students’ knowledge through content-rich non-fiction.', 'Once')
('4a. For general/self-contained/literacy/science/social studies: During your last instructional unit, how often were you able to build students’ knowledge through content-rich non-fiction.', 'Once a month')
('4a. For general/self-contained/literacy/science/social studies: During your last instructional unit, how often were you able to build students’ knowledge through content-rich non-fiction.', 'Weekly')
('4a. For general/self-contained/literacy/science/social studies: During your last instructional unit, how often were you able to build students’ knowledge through content-rich non-fiction.

Questions 4s and 5s have the following 6 options:  
'Never', 'Once', 'Once a month', 'Weekly', 'A few times a week', 'Almost every day'  

In [145]:
teacher_2015.loc[:1, teacher_2015.columns.get_level_values(1) == 'Rarely']

,"6a. In your first class today, how often do students build on each other’s ideas during class discussions?","6b. In your first class today, how often do students use data or text references to support their ideas?","6c. In your first class today, how often do students show that they respect each other’s ideas?","6d. In your first class today, how often do students provide constructive feedback to their peers/teachers?","6e. In your first class today, how often do most students participate in class discussions?","23a. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How often do students in your class(es) work quietly and calmly?","23b. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How often do students in your class(es) feel challenged?","23c. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How often do students in your class(es) listen carefully when the teacher gives directions?","23d. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How often do students in your class(es) have to work hard to do well?","23e. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How often do students in your class(es) follow the rules in class?","23f. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How often do students in your class(es) respond to challenging test questions?","23g. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How often do students in your class(es) pay attention when they are supposed to?","23h. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How often do students in your class(es) respond to challenging questions in class?","23i. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How often do students in your class(es) do their work when they are supposed to?","23j. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How often do students in your class(es) behave well in class even when the teacher isn't watching?","24a. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. Students are expected to think critically.","24b. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. Students learn a lot in this class.","24c. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. Students are expected to work hard.","24d. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. Students are expected to do their best all of the time.","24e. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. Students are expected to think, not just memorize things."
,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely,Rarely
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
for column in teacher_2015.columns:
    if re.match(r'6[a-z]', column[0]):
        print(column)

('6a. In your first class today, how often do students build on each other’s ideas during class discussions?', 'Never')
('6a. In your first class today, how often do students build on each other’s ideas during class discussions?', 'Rarely')
('6a. In your first class today, how often do students build on each other’s ideas during class discussions?', 'Sometimes ')
('6a. In your first class today, how often do students build on each other’s ideas during class discussions?', 'Often')
('6a. In your first class today, how often do students build on each other’s ideas during class discussions?', 'Almost always ')
('6a. In your first class today, how often do students build on each other’s ideas during class discussions?', 'Always ')
('6b. In your first class today, how often do students use data or text references to support their ideas?', 'Never')
('6b. In your first class today, how often do students use data or text references to support their ideas?', 'Rarely')
('6b. In your first class 

Questions 6s, 23s, and 24s have the following 6 options:  
'Never', 'Rarely', 'Sometimes ', 'Often', 'Almost always ', 'Always '

In [147]:
pd.set_option('display.max_columns', 500)

teacher_2015.loc[:1, teacher_2015.columns.get_level_values(1) == "I don't know "]

,8a. The School Leadership Team (SLT) at this school makes decisions that affect the whole school.,8b. The School Leadership Team (SLT) at this school provides resources to support the SLT’s priorities.,8c. The School Leadership Team (SLT) at this school assesses how our school’s educational programs affect student achievement.,8d. The School Leadership Team (SLT) at this school writes a high-quality Comprehensive Education Plan (CEP) for our school.,"12a. Overall, my professional development experiences this year have been sustained and coherently focused, rather than short-term and unrelated.","12b. Overall, my professional development experiences this year have included enough time to think carefully about, try, and evaluate new ideas.","12c. Overall, my professional development experiences this year have included opportunities to work productively with colleagues in my school.","12d. Overall, my professional development experiences this year have included opportunities to work productively with teachers from other schools.",13a. I am able to use my student's prior knowledge related to their cultural and linguistic backgrounds to help make learning meaningful.,13b. I am able to modify instructional activities and materials to meet the developmental needs and learning interests of all my students.,13c. I am able to critically examine the curriculum to determine whether it appropriately represents many groups.,13d. I am able to design appropriate instruction that is matched to English Language Learners (ELL) proficiency and students with disabilities.,13e. I am able to create a learning environment that reflects the various backgrounds of all of my students.,13f. I am able to identify cultural differences when communicating with parents/guardians regarding their child's educational progress.,13g. I am able to develop appropriate Individual Education Programs for my students with disabilities.,13h. I am able to distinguish linguistic/cultural differences from learning difficulties.,"19a. The following question is only for respondents who selected ""High (9-12)"" in question 18. Adults at this school support students in navigating the post-secondary process.","19b. The following question is only for respondents who selected ""High (9-12)"" in question 18. Adults at this school teach students the perseverance skills they need to succeed after high school.","19c. The following question is only for respondents who selected ""High (9-12)"" in question 18. Adults at this school instill academic self-confidence in students.","19d. The following question is only for respondents who selected ""High (9-12)"" in question 18. Adults at this school help students aspiring to enter the workforce, develop a plan to reach their employment goals.","19e. The following question is only for respondents who selected ""High (9-12)"" in question 18. Adults at this school teach critical thinking skills to students.","19f. The following question is only for respondents who selected ""High (9-12)"" in question 18. Adults at this school provide students with information about the college enrollment process.","19g. The following question is only for respondents who selected ""High (9-12)"" in question 18. Adults at this school inspire students to work towards a college degree.","19h. The following question is only for respondents who selected ""High (9-12)"" in question 18. Adults at this school teach students how to advocate for themselves.","19i. The following question is only for respondents who selected ""High (9-12)"" in question 18. Adults at this school support students to live independently.","19a. The following question is only for respondents who selected either ""Elementary (PreK-5)"" or ""Middle (6-8)"" in question 18. This school provides useful information to students about the application/enrollment process to middle or high school.","19b. The following question is only for respondents who selected either ""Elementary (PreK-5)"" or ""Middle

In [148]:
for column in teacher_2015.columns:
    if re.match(r'8[a-z]', column[0]):
        print(column)

('8a. The School Leadership Team (SLT) at this school makes decisions that affect the whole school. ', 'Strongly disagree')
('8a. The School Leadership Team (SLT) at this school makes decisions that affect the whole school. ', 'Disagree')
('8a. The School Leadership Team (SLT) at this school makes decisions that affect the whole school. ', 'Somewhat disagree')
('8a. The School Leadership Team (SLT) at this school makes decisions that affect the whole school. ', 'Somewhat agree')
('8a. The School Leadership Team (SLT) at this school makes decisions that affect the whole school. ', 'Agree')
('8a. The School Leadership Team (SLT) at this school makes decisions that affect the whole school. ', 'Strongly Agree')
('8a. The School Leadership Team (SLT) at this school makes decisions that affect the whole school. ', "I don't know ")
('8b. The School Leadership Team (SLT) at this school provides resources to support the SLT’s priorities. ', 'Strongly disagree')
('8b. The School Leadership Team 

Questions 8s, 12s, 13s, and 19s have the following 7 options:  
Strongly disagree', 'Disagree', 'Somewhat disagree', 'Somewhat agree', 'Agree', 'Strongly Agree', 'I don't know'

In [149]:
teacher_2015.loc[:1, teacher_2015.columns.get_level_values(1) == 'About half']

,"25a. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How many of the students in your class(es) feel it is important to come to school every day?","25b. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How many of the students in your class(es) feel it is important to be engaged in class?","25c. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How many of the students in your class(es) think doing homework is important?","25d. The following question is only for respondents who selected ""Elementary (PreK-5)"" in question 18. How many of the students in your class(es) try hard to get good grades?"
,About half,About half,About half,About half
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN


In [150]:
for column in teacher_2015.columns:
    if re.match(r'25[a-z]', column[0]):
        print(column)

('25a. The following question is only for respondents who selected "Elementary (PreK-5)" in question 18. How many of the students in your class(es) feel it is important to come to school every day?', 'None')
('25a. The following question is only for respondents who selected "Elementary (PreK-5)" in question 18. How many of the students in your class(es) feel it is important to come to school every day?', 'Some')
('25a. The following question is only for respondents who selected "Elementary (PreK-5)" in question 18. How many of the students in your class(es) feel it is important to come to school every day?', 'About half')
('25a. The following question is only for respondents who selected "Elementary (PreK-5)" in question 18. How many of the students in your class(es) feel it is important to come to school every day?', 'Most')
('25a. The following question is only for respondents who selected "Elementary (PreK-5)" in question 18. How many of the students in your class(es) feel it is imp

Questions 25s have the following 6 options:  
None, Some, About half, Most, Nearly all, All

In [151]:
for column in teacher_2015.columns:
    if re.match(r'2', column[0]):
        print(column)

('2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?', 'None')
('2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?', 'A few')
('2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?', 'Some')
('2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?', 'Most')
('2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?', 'Nearly all')
('2a. How many teachers at this school (not just in your practice) help maintain discipline in the entire school?', 'All')
('2b. How many teachers at this school (not just in your practice) are really trying to improve their teaching?', 'None')
('2b. How many teachers at this school (not just in your practice) are really trying to improve their teaching?', 'A few')
('2b.

There are no 3s and 7s, but he rest of them appear to have the following 6 options:   
Strongly disagree, Disagree, Somewhat disagree, Somewhat agree, Agree, Strongly agree

In [152]:
# grab the first part where it has the question number right before the .
questions = teacher_2015.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+[a-z]*)\.(.+)', question).group(1) for question in questions]

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if q_num.startswith(r'18'):
        for i in range(1, 4):
            new_cols.append(q_num + '_' + str(i))
    elif re.match(r'8[a-z]|12[a-z]|13[a-z]|19[a-z]', q_num):
        for i in [1, 2, 3, 4, 5, 6, 0]:
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 7):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'parent_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

# change column names
teacher_2015.columns = new_cols

In [153]:
teacher_2016.head(2)

DBN                School Name Total Teacher Response Rate  \
  Unnamed: 0_level_1         Unnamed: 1_level_1          Unnamed: 2_level_1   
0             01M015  P.S. 015 Roberto Clemente                        0.96   
1             01M019        P.S. 019 Asher Levy                        0.90   

  1a. How many teachers at this school help maintain discipline in the entire school, not just their classroom?  \
                                                                                                           None   
0                                                0.0                                                              
1                                                0.0                                                              

                     \
   Some A lot   All   
0  0.05  0.36  0.59   
1  0.17  0.39  0.44   

  1b. How many teachers at this school are really trying to improve their teaching?  \
                                                                               None   
0                                                0.0                                  
1                                                0.0                                  

                     \
   Some A lot   All   
0  0.00  0.27  0.73   
1  0.06  0.50  0.44   

  1c. How many teachers at this school take responsibility for improving the school?  \
                                                                                None   
0                                                0.0                                   
1                                                0.0                                   

                     \
   Some A lot   All   
0  0.00  0.23  0.77   
1  0.14  0.50  0.36   

  1d. How many teachers at this school feel responsible for helping students develop self-management?  \
                                                                                                 None   
0                                                0.0                                                    
1                                                0.0                                                    

                     \
   Some A lot   All   
0  0.00  0.27  0.73   
1  0.08  0.47  0.44   

  1e. How many teachers at this school are willing to take risks to make the school better?  \
                                                                                       None   
0                                               0.00                                          
1                                               0.03                                          

                     \
   Some A lot   All   
0  0.00  0.36  0.64   
1  0.28  0.31  0.39   

  1f. How many teachers at this school are eager to try new ideas?        \
                                                              None  Some   
0                                                0.0                0.00   
1                                                0.0                0.11   

               \
  A lot   All   
0  0.29  0.71   
1  0.61  0.28   

  1g. How many teachers at this school feel responsible that all students learn?  \
                                                                            None   
0                                                0.0                               
1                                                0.0                               

                     \
   Some A lot   All   
0  0.00  0.27  0.73   
1  0.06  0.39  0.56   

  2a.  I am able to use my students' prior knowledge related to their cultural and linguistic backgrounds to help make learning meaningful.  \
                                                                                                                          Strongly disagree   
0                                                0.0                                                                                          
1                                    

In [154]:
teacher_2016.shape

(3098, 529)

In [155]:
teacher_2016.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'None', 'Some', 'A lot', 'All', 'Strongly disagree', 'Disagree',
       'Agree', 'Strongly agree', 'I don't know', 'No influence', 'Little',
       'A moderate amount', 'A great deal of influence', 'Very dissatisfied',
       'Dissatisfied', 'Satisfied', 'Very Satisfied', 'Strongly Agree',
       'None of the time', 'Some of the time', 'Most of the time',
       'All of the time'],
      dtype='object')

In [156]:
teacher_2016.loc[:2, teacher_2016.columns.get_level_values(1) == "I don't know"]

,2a. I am able to use my students' prior knowledge related to their cultural and linguistic backgrounds to help make learning meaningful.,2b. I am able to modify instructional activities and materials to meet the developmental needs and learning interests of all my students.,2c. I am able to critically examine the curriculum to determine whether it appropriately represents all groups.,2d. I am able to design appropriate instruction that is matched to English language learners (ELL) proficiency and students with disabilities.,2e. I am able to identify cultural differences when communicating with parents/guardians regarding their child's educational progress.,2f. I am able to develop appropriate Individual Education Programs for my students with disabilities.,2g. I am able to distinguish linguistic/cultural differences from learning difficulties.,"14a. The performance of the citywide Panel for Educational Policy with regard to school resources, oversight, curriculum, and progress in student achievement.","14b. The performance of the Schools Chancellor with regard to school resources, oversight, curriculum, and progress in student achievement.",20a. How many adults at this school help students develop the skills they need to complete challenging coursework despite obstacles?,20b. How many adults at this school tell their students they believe they can achieve high academic standards?,20c. How many adults at this school teach critical thinking skills to students?,20d. How many adults at this school teach students how to advocate for themselves?,20e. How many adults at this school teach students the organizational skills needed to be prepared for their next level?,20f. How many adults at this school recognize disruptive behavior as social-emotional learning opportunities?,"20g. How many adults at this school teach students the skills they need to regulate their behavior (i.e. by focusing their attention, controlling their emotions, or managing their thinking, behavior, and feelings)?",20h. How many adults at this school have access to school based supports to assist in behavioral/emotional escalations?,"26a. If you are a teacher in grades K-8, ANSWER this question. It’s a priority at this school that adults provide useful information to students about the application/enrollment process to middle or high school.","26b. If you are a teacher in grades K-8, ANSWER this question. It’s a priority at this school that adults provide guidance for the application process to middle or high school.","26c. If you are a teacher in grades K-8, ANSWER this question. It’s a priority at this school that adults educate families about the application/enrollment process for middle or high school.","27a. If you are a teacher in grades 9-12, ANSWER this question. It’s a priority at this school that adults provide students with information about the college enrollment process.","27b. If you are a teacher in grades 9-12, ANSWER this question. It’s a priority at this school that adults help students plan for their next steps after graduation (career planning, college selection and application process, financial aid process, etc.).","27c. If you are a teacher in grades 9-12, ANSWER this question. It’s a priority at this school that adults help students choose which colleges to apply to.","27d. If you are a teacher in grades 9-12, ANSWER this question. It’s a priority at this school that adults talk to students about how to pay for college.","27e. If you are a teacher in grades 9-12, ANSWER this question. It’s a priority at this school that adults help students plan for how to meet their future career goals.","28a. If you are a teacher in grades 9-12, ANSWER this question. How many adults at this school create an atmosphere that encourages students to work towards a college degree?"
,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't k

In [157]:
for column in teacher_2016.columns:
    if re.match(r'2[a-z]', column[0]):
        print(column)

("2a.  I am able to use my students' prior knowledge related to their cultural and linguistic backgrounds to help make learning meaningful.", 'Strongly disagree')
("2a.  I am able to use my students' prior knowledge related to their cultural and linguistic backgrounds to help make learning meaningful.", 'Disagree')
("2a.  I am able to use my students' prior knowledge related to their cultural and linguistic backgrounds to help make learning meaningful.", 'Agree')
("2a.  I am able to use my students' prior knowledge related to their cultural and linguistic backgrounds to help make learning meaningful.", 'Strongly agree')
("2a.  I am able to use my students' prior knowledge related to their cultural and linguistic backgrounds to help make learning meaningful.", "I don't know")
('2b.  I am able to modify instructional activities and materials to meet the developmental needs and learning interests of all my students.', 'Strongly disagree')
('2b.  I am able to modify instructional activitie

Questions 20s, 28s have 5 answer options:   
None, Some, A lot, All, I don't know   
Questions 2s, 26s, 27s have 5 answer options:
Strongly disagree, Disagree, Agree, Strongly Agree, I don't know   
Questions 14s have 5 answer options:   
Very dissatisfied, Dissatisfied, Satisfied, Very satisfied, I don't know

In [158]:
teacher_2016.loc[:2, teacher_2016.columns.get_level_values(1) == 'None of the time']

,"22a. If you are a teacher of grades K-5, ANSWER this question. How often do students in your class(es) work in small groups?"
,None of the time
0,0.05
1,0.00
2,0.00


In [159]:
for column in teacher_2016.columns:
    if re.match(r'22[a-z]*', column[0]):
        print(column)

('22a. If you are a teacher of grades K-5, ANSWER this question. How often do students in your class(es) work in small groups?', 'None of the time')
('22a. If you are a teacher of grades K-5, ANSWER this question. How often do students in your class(es) work in small groups?', 'Some of the time')
('22a. If you are a teacher of grades K-5, ANSWER this question. How often do students in your class(es) work in small groups?', 'Most of the time')
('22a. If you are a teacher of grades K-5, ANSWER this question. How often do students in your class(es) work in small groups?', 'All of the time')


Question 22 has 4 answer options:  
None of the time, Some of the time, Most of the time, All of the time

In [160]:
teacher_2016.loc[:2, teacher_2016.columns.get_level_values(1) == 'No influence']

,13a. Hiring new professional personnel.,13b. Planning how discretionary school funds should be used.,13c. Selecting instructional materials used in classrooms.,13d. Developing instructional materials.,13e. Setting standards for student behavior.
,No influence,No influence,No influence,No influence,No influence
0,0.05,0.05,0.10,0.09,0.00
1,0.31,0.25,0.03,0.00,0.00
2,0.44,0.60,0.32,0.32,0.39


In [161]:
for column in teacher_2016.columns:
    if re.match(r'8[a-z]', column[0]):
        print(column)

('8a. At this school once we start a new program, we follow up to make sure that it’s working.', 'Strongly disagree')
('8a. At this school once we start a new program, we follow up to make sure that it’s working.', 'Disagree')
('8a. At this school once we start a new program, we follow up to make sure that it’s working.', 'Agree')
('8a. At this school once we start a new program, we follow up to make sure that it’s working.', 'Strongly agree')
('8b. At this school it is clear how all of the programs offered are connected to our school’s instructional vision.', 'Strongly disagree')
('8b. At this school it is clear how all of the programs offered are connected to our school’s instructional vision.', 'Disagree')
('8b. At this school it is clear how all of the programs offered are connected to our school’s instructional vision.', 'Agree')
('8b. At this school it is clear how all of the programs offered are connected to our school’s instructional vision.', 'Strongly agree')
('8c At this sch

Questions 13s have 4 answer options:  
No influence, Little, A moderate amount, A great deal of influence  
The rest of the questions have 4 options.

In [162]:
# grab the first part where it has the question number right before the .
questions = teacher_2016.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+[a-z]*)\.|(\d+[a-z]*)\s', question).group(1) for question in questions]
q_nums = ['8c' if x == None else x for x in q_nums] # need to fix this since, there is no '.' after question number

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if re.match(r'2[a-z]|14[a-z]|20[a-z]|26[a-z]|27[a-z]|28[a-z]', q_num):
        for i in [1, 2, 3, 4, 0]:
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'parent_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

# change column names
teacher_2016.columns = new_cols

In [163]:
teacher_2017.head(2)

DBN                School Name Total Teacher Response Rate  \
  Unnamed: 0_level_1         Unnamed: 1_level_1          Unnamed: 2_level_1   
0             01M015  P.S. 015 Roberto Clemente                         100   
1             01M019        P.S. 019 Asher Levy                         100   

  1a. How many teachers at this school help maintain discipline in the entire school, not just their classroom?  \
                                                                                                           None   
0                                                0.0                                                              
1                                                2.0                                                              

                     \
   Some A lot   All   
0   0.0  42.0  58.0   
1  17.0  33.0  48.0   

  1b. How many teachers at this school are actively trying to improve their teaching?  \
                                                                                 None   
0                                                0.0                                    
1                                                0.0                                    

                    \
  Some A lot   All   
0  4.0  13.0  83.0   
1  5.0  40.0  55.0   

  1c. How many teachers at this school take responsibility for improving the school?  \
                                                                                None   
0                                                0.0                                   
1                                                0.0                                   

                     \
   Some A lot   All   
0   8.0  17.0  75.0   
1  19.0  26.0  55.0   

  1d. How many teachers at this school are eager to try new ideas?             \
                                                              None Some A lot   
0                                                0.0                0.0  29.0   
1                                                0.0                5.0  48.0   

         \
    All   
0  71.0   
1  48.0   

  1e. How many teachers at this school feel responsible that all students learn?  \
                                                                            None   
0                                                0.0                               
1                                                0.0                               

                    \
  Some A lot   All   
0  4.0  17.0  79.0   
1  5.0  33.0  62.0   

  2a. I am able to receive support around how to incorporate students' cultural and linguistic backgrounds in my practice.  \
                                                                                                         Strongly Disagree   
0                                                4.0                                                                         
1                                               10.0                                                                         

                                              \
  Disagree Agree Strongly Agree I don't know   
0      0.0  29.0           67.0          0.0   
1      0.0  24.0           64.0          2.0   

  2b.  I am able to use my students' prior knowledge to make my lessons relevant to their everyday life.  \
                                                                                       Strongly disagree   
0                                                4.0                                                       
1                                                5.0                                                       

                                              \
  Disagree Agree Strongly agree I don't know   
0      0.0  13.0           83.0          0.0   
1      0.0  20.0           76.0          0.0   

  2c.  I am able to modify instructional activities and materials to meet the developmental needs and learning interests of all my stu

In [164]:
teacher_2017.shape

(1904, 561)

In [165]:
teacher_2017.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'None', 'Some', 'A lot', 'All', 'Strongly Disagree', 'Disagree',
       'Agree', 'Strongly Agree', 'I don't know', 'Strongly disagree',
       'Strongly agree', 'None of the time', 'Rarely', 'Some of the time',
       'Most of the time', 'No influence', 'Little', 'A moderate amount',
       'A great deal of influence', 'Very dissatisfied', 'Dissatisfied',
       'Satisfied', 'Very Satisfied', 'Social Studies', 'Science', 'ELA',
       'Math', 'All subjects (e.g., elementary, self-contained)', 'Other',
       'N/A', 'Elementary (PreK-5)', 'Middle (6-8)', 'High (9-12)'],
      dtype='object')

In [166]:
teacher_2017.loc[:1, teacher_2017.columns.get_level_values(1) == "I don't know"]

,2a. I am able to receive support around how to incorporate students' cultural and linguistic backgrounds in my practice.,2b. I am able to use my students' prior knowledge to make my lessons relevant to their everyday life.,2c. I am able to modify instructional activities and materials to meet the developmental needs and learning interests of all my students.,2d. I am able to adapt instruction to ensure it represents all cultures/backgrounds positively.,2e. I am able to design appropriate instruction that is matched to students' need (i.e. English language learners (ELL) proficiency and students with disabilities).,2f. I am able to apply my knowledge of parents' various cultural backgrounds when collaborating with them regarding their child's educational progress.,2g. I am able to develop appropriate Individual Education Programs for my students with disabilities.,2h. I am able to distinguish linguistic/cultural differences from learning difficulties.,"13a. The performance of the citywide Panel for Educational Policy with regard to school resources, oversight, curriculum, and progress in student achievement.","13b. The performance of the Schools Chancellor with regard to school resources, oversight, curriculum, and progress in student achievement.",20a. How many adults at this school help students develop the skills they need to complete challenging coursework despite obstacles?,20b. How many adults at this school tell their students they believe they can achieve high academic standards?,20c. How many adults at this school teach critical thinking skills to students?,20d. How many adults at this school teach students how to advocate for themselves?,20e. How many adults at this school teach students the organizational skills needed to be prepared for their next level?,20f. How many adults at this school recognize disruptive behavior as social-emotional learning opportunities?,"20g. How many adults at this school teach students the skills they need to regulate their behavior (i.e. by focusing their attention, controlling their emotions, or managing their thinking, behavior, and feelings)?",20h. How many adults at this school have access to school based supports to assist in behavioral/emotional escalations?,25a. It’s a priority at this school that adults provide useful information to students about the application/enrollment process to middle or high school.,25b. It’s a priority at this school that adults provide guidance for the application process to middle or high school.,25c. It’s a priority at this school that adults educate families about the application/enrollment process for middle or high school.,26a. Adults at this school create an atmosphere that encourages students to work towards a college degree?,"26b. Adults at this school provide students with information about the college enrollment process (college selection and application process, financial aid process, course registration).",26c. Adults at this school help students consider which colleges to apply to.,"26d. Adults at this school show students options for how to pay for college (scholarship, grants, loans, work study programs).",26e. Adults at this school help students plan for how to meet their future career goals.
,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know,I don't know
0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,54.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,13.0,9.0,NaN,NaN,NaN,NaN,NaN
1,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,37.0,21.0,5.0,2.0,2.0,5.0,2.0,2.0,0.0,2.0,19.0,12.0,12.0,NaN,NaN,NaN,NaN,NaN


In [167]:
for column in teacher_2017.columns:
    if re.match(r'26[a-z]', column[0]):
        print(column)

('26a. Adults at this school create an atmosphere that encourages students to work towards a college degree?', 'Strongly disagree')
('26a. Adults at this school create an atmosphere that encourages students to work towards a college degree?', 'Disagree')
('26a. Adults at this school create an atmosphere that encourages students to work towards a college degree?', 'Agree')
('26a. Adults at this school create an atmosphere that encourages students to work towards a college degree?', 'Strongly agree')
('26a. Adults at this school create an atmosphere that encourages students to work towards a college degree?', "I don't know")
('26b. Adults at this school provide students with information about the college enrollment process (college selection and application process, financial aid process, course registration).', 'Strongly disagree')
('26b. Adults at this school provide students with information about the college enrollment process (college selection and application process, financial aid

Questions 2s, 25s, 26s have 5 options:  
Strongly disagree, Disagree, Agree, Strongly agree, I don't know   
Questions 13s have 5 options:   
Very dissatisfied, dissatisfied, Satisfied, Very Satisfied, I don't know  
Questions 20s have 5 options:   
None, Some, A lot, All, I don't know

In [168]:
teacher_2017.loc[:1, teacher_2017.columns.get_level_values(1) == 'No influence']

,12a. How much influence do teachers have over school policy in hiring new professional personnel?,12b. How much influence do teachers have over school policy in planning how discretionary school funds should be used?,12c. How much influence do teachers have over school policy in selecting instructional materials used in classrooms?,12d. How much influence do teachers have over school policy in developing instructional materials?,12e. How much influence do teachers have over school policy in setting standards for student behavior?
,No influence,No influence,No influence,No influence,No influence
0,0.0,4.0,0.0,0.0,0.0
1,24.0,26.0,5.0,2.0,2.0


In [169]:
for column in teacher_2017.columns:
    if re.match(r'12[a-z]', column[0]):
        print(column)

('12a. How much influence do teachers have over school policy in hiring new professional personnel?', 'No influence')
('12a. How much influence do teachers have over school policy in hiring new professional personnel?', 'Little')
('12a. How much influence do teachers have over school policy in hiring new professional personnel?', 'A moderate amount')
('12a. How much influence do teachers have over school policy in hiring new professional personnel?', 'A great deal of influence')
('12b. How much influence do teachers have over school policy in planning how discretionary school funds should be used?', 'No influence')
('12b. How much influence do teachers have over school policy in planning how discretionary school funds should be used?', 'Little')
('12b. How much influence do teachers have over school policy in planning how discretionary school funds should be used?', 'A moderate amount')
('12b. How much influence do teachers have over school policy in planning how discretionary school f

Questions 12s have 4 options:  
No influence, Little, A moderate amount, A great deal of influence  

In [170]:
teacher_2017.loc[:1, teacher_2017.columns.get_level_values(1) == 'Other']

,14. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach?
,Other
0,24.0
1,19.0


In [171]:
for column in teacher_2017.columns:
    if re.match(r'14', column[0]):
        print(column)

('14. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'Social Studies')
('14. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'Science')
('14. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'ELA')
('14. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'Math')
('14. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'All subjects (e.g., elementary, self-contained)')
('14. This question is for informational purposes 

Question 14 has 6 options:   
Social Studies, Science, ELA, Math, All subjects (e.g., elementary, self-contained), Other  

In [172]:
teacher_2017.loc[:1, teacher_2017.columns.get_level_values(1) == 'N/A']

,"17a. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students' knowledge through content rich non-fiction.","17b. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for reading and writing experiences grounded in evidence from text, both literary and informational.","17c. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for students to interact with complex grade-level text.","17d. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for students to interact with academic language.","17e. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for focusing deeply on the concepts emphasized in the standards to help students build strong foundations for learning.","17f. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for creating coherent progressions within the standards from previous grades to current grade so student knowledge/skills build onto previous learning as foundations for math concepts.","17g. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for developing students' conceptual understanding, procedural fluency, and their ability to apply math in context."
,N/A,N/A,N/A,N/A,N/A,N/A,N/A
0,0.0,0.0,0.0,0.0,0.0,5.0,5.0
1,8.0,8.0,8.0,5.0,3.0,11.0,13.0


In [173]:
for column in teacher_2017.columns:
    if re.match(r'17[a-z]', column[0]):
        print(column)

("17a. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students' knowledge through content rich non-fiction.", 'Strongly disagree')
("17a. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students' knowledge through content rich non-fiction.", 'Disagree')
("17a. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students' knowledge through content rich non-fiction.", 'Agree')
("17a. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students' knowledge through content rich non-fiction.", 'Strongly Agree')
("17a. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students' knowledge through content rich non-fictio

Questions 17s have 5 options:  
Strongly disagree, Disagree, Agree, Strongly Agree', N/A 

In [174]:
teacher_2017.loc[:1, teacher_2017.columns.get_level_values(1) == 'Elementary (PreK-5)'] 

,21. This question is for informational purposes only. What grades do you primarily teach?
,Elementary (PreK-5)
0,96.0
1,100.0


In [175]:
for column in teacher_2017.columns:
    if re.match(r'21', column[0]):
        print(column)

('21. This question is for informational purposes only. What grades do you primarily teach?', 'Elementary (PreK-5)')
('21. This question is for informational purposes only. What grades do you primarily teach?', 'Middle (6-8)')
('21. This question is for informational purposes only. What grades do you primarily teach?', 'High (9-12)')


Question 21 has 3 options:  
Elementary (PreK-5), Middle (6-8), High (9-12)  
The rest have 4 options

In [176]:
# grab the first part where it has the question number right before the .
questions = teacher_2017.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+[a-z]*)\.', question).group(1) for question in questions]

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if re.match(r'2[a-z]|13[a-z]|17[a-z]|20[a-z]|25[a-z]|26[a-z]', q_num):
        for i in [1, 2, 3, 4, 0]:
            new_cols.append(q_num + '_' + str(i))
    elif re.match(r'21', q_num):
        for i in range(1, 4):
            new_cols.append(q_num + '_' + str(i))
    elif re.match(r'14', q_num):
        for i in range(1, 7):
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'parent_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

# change column names
teacher_2017.columns = new_cols

In [177]:
teacher_2018.head(2)

DBN                School Name Total Teacher Response Rate  \
  Unnamed: 0_level_1         Unnamed: 1_level_1          Unnamed: 2_level_1   
0             01M015  P.S. 015 ROBERTO CLEMENTE                         100   
1             01M019        P.S. 019 ASHER LEVY                          95   

  1a. How many teachers at this school help maintain discipline in the entire school, not just their classroom?  \
                                                                                                           None   
0                                                0.0                                                              
1                                                0.0                                                              

                     \
   Some A lot   All   
0   4.0  44.0  52.0   
1  15.0  53.0  33.0   

  1b. How many teachers at this school are actively trying to improve their teaching?  \
                                                                                 None   
0                                                0.0                                    
1                                                0.0                                    

                     \
   Some A lot   All   
0   4.0  33.0  63.0   
1  15.0  38.0  48.0   

  1c. How many teachers at this school take responsibility for improving the school?  \
                                                                                None   
0                                                0.0                                   
1                                                3.0                                   

                     \
   Some A lot   All   
0   4.0  36.0  60.0   
1  10.0  50.0  38.0   

  1d. How many teachers at this school are eager to try new ideas?        \
                                                              None  Some   
0                                                0.0                12.0   
1                                                0.0                13.0   

               \
  A lot   All   
0  32.0  56.0   
1  55.0  33.0   

  1e. How many teachers at this school feel responsible that all students learn?  \
                                                                            None   
0                                                0.0                               
1                                                0.0                               

                     \
   Some A lot   All   
0   8.0  28.0  64.0   
1  10.0  33.0  58.0   

  2a. I am able to receive support around how to incorporate students' cultural and linguistic backgrounds in my practice.  \
                                                                                                         Strongly Disagree   
0                                                0.0                                                                         
1                                                5.0                                                                         

                                              \
  Disagree Agree Strongly Agree I don't know   
0      8.0  32.0           56.0          4.0   
1     10.0  38.0           48.0          0.0   

  2b.  I am able to use my students' prior knowledge to make my lessons relevant to their everyday life.  \
                                                                                       Strongly disagree   
0                                                0.0                                                       
1                                                5.0                                                       

                                              \
  Disagree Agree Strongly agree I don't know   
0      4.0  24.0           72.0          0.0   
1      0.0  28.0           68.0          0.0   

  2c.  I am able to modify instructional activities and materials to meet the developmental needs and learning interests o

In [178]:
teacher_2018.shape

(1902, 591)

In [179]:
teacher_2018.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'None', 'Some', 'A lot', 'All', 'Strongly Disagree', 'Disagree',
       'Agree', 'Strongly Agree', 'I don't know', 'Strongly disagree',
       'Strongly agree', 'None of the time', 'Rarely', 'Some of the time',
       'Most of the time', 'No influence', 'Little', 'A moderate amount',
       'A great deal of influence', 'Very dissatisfied', 'Dissatisfied',
       'Satisfied', 'Very Satisfied', 'Early childhood (e.g. Pre-K or 3-K)',
       'Social Studies', 'Science', 'ELA', 'Math',
       'All subjects (e.g., elementary, self-contained)', 'Other', 'N/A',
       'I don’t know', 'I don’t know ', 'Early Childhood (Pre-K or 3-K)',
       ' Elementary (K-5)', 'Middle (6-8)', 'High (9-12)'],
      dtype='object')

In [180]:
teacher_2018.loc[:1, teacher_2018.columns.get_level_values(1) == "I don’t know "]

,21d. How many adults at this school teach students how to advocate for themselves?,21e. How many adults at this school teach students the organizational skills needed to be prepared for their next level?,21f. How many adults at this school recognize disruptive behavior as social-emotional learning opportunities?,"21g. How many adults at this school teach students the skills they need to regulate their behavior (i.e. by focusing their attention, controlling their emotions, or managing their thinking, behavior, and feelings)?",21h. How many adults at this school have access to school based supports to assist in behavioral/emotional escalations?
,I don’t know,I don’t know,I don’t know,I don’t know,I don’t know
0,0.0,4.0,0.0,4.0,4.0
1,8.0,5.0,5.0,3.0,8.0


In [181]:
for column in teacher_2018.columns:
    if re.match(r'2[a-z]', column[0]):
        print(column)

("2a. I am able to receive support around how to incorporate students' cultural and linguistic backgrounds in my practice.", 'Strongly Disagree')
("2a. I am able to receive support around how to incorporate students' cultural and linguistic backgrounds in my practice.", 'Disagree')
("2a. I am able to receive support around how to incorporate students' cultural and linguistic backgrounds in my practice.", 'Agree')
("2a. I am able to receive support around how to incorporate students' cultural and linguistic backgrounds in my practice.", 'Strongly Agree')
("2a. I am able to receive support around how to incorporate students' cultural and linguistic backgrounds in my practice.", "I don't know")
("2b.  I am able to use my students' prior knowledge to make my lessons relevant to their everyday life.", 'Strongly disagree')
("2b.  I am able to use my students' prior knowledge to make my lessons relevant to their everyday life.", 'Disagree')
("2b.  I am able to use my students' prior knowledge

Questions 2s, 25s, 26s have 5 options:  
Strongly disagree, Disagree, Agree, Strongly agree, I don't know   
Questions 13s have 5 options:   
Very dissatisfied, dissatisfied, Satisfied, Very Satisfied, I don't know  
Questions 21s have 5 options:   
None, Some, A lot, All, I don't know

In [182]:
teacher_2018.loc[:1, teacher_2018.columns.get_level_values(1) == 'Middle (6-8)']

,22. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)
,Middle (6-8)
0,0.0
1,0.0


In [183]:
for column in teacher_2018.columns:
    if re.match(r'22', column[0]):
        print(column)

('22. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)', 'Early Childhood (Pre-K or 3-K)')
('22. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)', ' Elementary (K-5)')
('22. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)', 'Middle (6-8)')
('22. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)', 'High (9-12)')


Question 22 has 4 options:   
Early Childhood (Pre-K or 3-K), Elementary (K-5), Middle (6-8), High (9-12)  

In [184]:
teacher_2018.loc[:1, teacher_2018.columns.get_level_values(1) == 'N/A']

,"18a. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students’ knowledge through content-rich non-fiction.","18b. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for reading and writing experiences grounded in evidence from text, both literary and informational.","18c. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for students to interact with complex grade-level text.","18d. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for students to interact with academic language.","18e. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for focusing deeply on the concepts emphasized in the standards to help students build strong foundations for learning.","18f. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for creating coherent progressions within the standards from previous grades to current grade so student knowledge/skills build onto previous learning as foundations for math concepts.","18g. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for developing students’ conceptual understanding, procedural fluency, and their ability to apply math in context."
,N/A,N/A,N/A,N/A,N/A,N/A,N/A
0,8.0,8.0,8.0,8.0,8.0,25.0,17.0
1,11.0,9.0,9.0,6.0,3.0,9.0,9.0


In [185]:
for column in teacher_2018.columns:
    if re.match(r'18[a-z]', column[0]):
        print(column)

('18a.  In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students’ knowledge through content-rich non-fiction.', 'Strongly Disagree')
('18a.  In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students’ knowledge through content-rich non-fiction.', 'Disagree')
('18a.  In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students’ knowledge through content-rich non-fiction.', 'Agree')
('18a.  In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students’ knowledge through content-rich non-fiction.', 'Strongly Agree')
('18a.  In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students’ knowledge through content-rich non-f

Questions 18s have 5 options:  
Strongly Disagree, Disagree, Agree, Strongly Agree, N/A

In [186]:
teacher_2018.loc[:1, teacher_2018.columns.get_level_values(1) == 'Social Studies']

,14. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach?
,Social Studies
0,24.0
1,10.0


In [187]:
for column in teacher_2018.columns:
    if re.match(r'14', column[0]):
        print(column)

('14. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'Early childhood (e.g. Pre-K or 3-K)')
('14. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'Social Studies')
('14. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'Science')
('14. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'ELA')
('14. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'Math')
('14. This question is for informational purposes only. It wil

Question 14 has 7 options:   
Early childhood (e.g. Pre-K or 3-K), Social Studies, ELA, Math, All subjects (e.g., elementary, self-contained), 'Other'

In [188]:
teacher_2018.loc[:1, teacher_2018.columns.get_level_values(1) == 'A great deal of influence'] 

,12a. Hiring new professional personnel.,12b. Planning how discretionary school funds should be used.,12c. Selecting instructional materials used in classrooms.,12d. Developing instructional materials.,12e. Setting standards for student behavior.
,A great deal of influence,A great deal of influence,A great deal of influence,A great deal of influence,A great deal of influence
0,28.0,4.0,29.0,46.0,42.0
1,10.0,8.0,28.0,38.0,48.0


In [189]:
for column in teacher_2018.columns:
    if re.match(r'12', column[0]):
        print(column)

('12a. Hiring new professional personnel.', 'No influence')
('12a. Hiring new professional personnel.', 'Little')
('12a. Hiring new professional personnel.', 'A moderate amount')
('12a. Hiring new professional personnel.', 'A great deal of influence')
('12b. Planning how discretionary school funds should be used.', 'No influence')
('12b. Planning how discretionary school funds should be used.', 'Little')
('12b. Planning how discretionary school funds should be used.', 'A moderate amount')
('12b. Planning how discretionary school funds should be used.', 'A great deal of influence')
('12c. Selecting instructional materials used in classrooms.', 'No influence')
('12c. Selecting instructional materials used in classrooms.', 'Little')
('12c. Selecting instructional materials used in classrooms.', 'A moderate amount')
('12c. Selecting instructional materials used in classrooms.', 'A great deal of influence')
('12d. Developing instructional materials.', 'No influence')
('12d. Developing instr

Questions 12s have 4 options:  
No influence, Little, A moderate amount, A great deal of influence   
The rest of the questions also appear to have 4 options.

In [190]:
# grab the first part where it has the question number right before the .
questions = teacher_2018.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+[a-z]*)\.', question).group(1) for question in questions]

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if re.match(r'2[a-z]|13[a-z]|18[a-z]|21[a-z]|27[a-z]|28[a-z]', q_num):
        for i in [1, 2, 3, 4, 0]:
            new_cols.append(q_num + '_' + str(i))
    elif re.match(r'14', q_num):
        for i in range(1, 8):
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'parent_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

# change column names
teacher_2018.columns = new_cols

In [191]:
teacher_2019.head(2)

DBN                School Name Total Teacher Response Rate  \
  Unnamed: 0_level_1         Unnamed: 1_level_1          Unnamed: 2_level_1   
0             01M015  P.S. 015 ROBERTO CLEMENTE                         100   
1             01M019        P.S. 019 ASHER LEVY                          93   

  1a. How many teachers at this school help maintain discipline in the entire school, not just their classroom?  \
                                                                                                           None   
0                                                0.0                                                              
1                                                0.0                                                              

                     \
   Some A lot   All   
0  23.0  50.0  39.0   
1  23.0  47.0  28.0   

  1b. How many teachers at this school are actively trying to improve their teaching?  \
                                                                                 None   
0                                                0.0                                    
1                                                0.0                                    

                    \
  Some A lot   All   
0  7.0  39.0  54.0   
1  8.0  57.0  35.0   

  1c. How many teachers at this school take responsibility for improving the school?  \
                                                                                None   
0                                                0.0                                   
1                                                0.0                                   

                     \
   Some A lot   All   
0  11.0  39.0  50.0   
1  30.0  38.0  32.0   

  1d. How many teachers at this school are eager to try new ideas?        \
                                                              None  Some   
0                                                0.0                14.0   
1                                                0.0                16.0   

               \
  A lot   All   
0  46.0  39.0   
1  54.0  30.0   

  1e. How many teachers at this school feel responsible that all students learn?  \
                                                                            None   
0                                                0.0                               
1                                                0.0                               

                     \
   Some A lot   All   
0   7.0  32.0  61.0   
1  11.0  30.0  59.0   

  2a. I am able to receive support around how to incorporate students' cultural and linguistic backgrounds in my practice.  \
                                                                                                         Strongly Disagree   
0                                                0.0                                                                         
1                                                0.0                                                                         

                                              \
  Disagree Agree Strongly Agree I don't know   
0      7.0  43.0           43.0          7.0   
1      3.0  30.0           68.0          0.0   

  2b.  I am able to use my students' prior knowledge to make my lessons relevant to their everyday life.  \
                                                                                       Strongly disagree   
0                                                0.0                                                       
1                                                0.0                                                       

                                              \
  Disagree Agree Strongly agree I don't know   
0      0.0  18.0           79.0          4.0   
1      0.0  24.0           76.0          0.0   

  2c.  I am able to modify instructional activities and materials to meet the developmental needs and learning interests of al

In [192]:
teacher_2019.shape

(1903, 579)

In [193]:
teacher_2019.columns.get_level_values(1).unique()

Index(['Unnamed: 0_level_1', 'Unnamed: 1_level_1', 'Unnamed: 2_level_1',
       'None', 'Some', 'A lot', 'All', 'Strongly Disagree', 'Disagree',
       'Agree', 'Strongly Agree', 'I don't know', 'Strongly disagree',
       'Strongly agree', 'None of the time', 'Rarely', 'Some of the time',
       'Most of the time', 'No influence', 'Little', 'A moderate amount',
       'A great deal of influence', 'Very dissatisfied', 'Dissatisfied',
       'Satisfied', 'Very Satisfied', 'Early childhood (e.g. Pre-K or 3-K)',
       'Social Studies', 'Science', 'ELA', 'Math',
       'All subjects (e.g., elementary, self-contained)', 'Other', 'N/A',
       'I don’t know', 'I don’t know ', 'Early Childhood (Pre-K or 3-K)',
       ' Elementary (K-5)', 'Middle (6-8)', 'High (9-12)'],
      dtype='object')

In [194]:
teacher_2019.loc[:1, teacher_2019.columns.get_level_values(1) == 'High (9-12)'] 

,24. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)
,High (9-12)
0,0.0
1,0.0


In [195]:
for column in teacher_2019.columns:
    if re.match(r'24', column[0]):
        print(column)

('24. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)', 'Early Childhood (Pre-K or 3-K)')
('24. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)', ' Elementary (K-5)')
('24. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)', 'Middle (6-8)')
('24. This question is for informational purposes only. What grades do you primarily teach? (Please mark one)', 'High (9-12)')


Question 24 has 4 options:  
Early Childhood (Pre-K or 3-K), Elemenbary (K-5), Middle (6-8), High (9-12)

In [196]:
teacher_2019.loc[:1, teacher_2019.columns.get_level_values(1) == 'Other'] 

,16. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach?
,Other
0,21.0
1,16.0


In [197]:
for column in teacher_2019.columns:
    if re.match(r'16', column[0]):
        print(column)

('16. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'Early childhood (e.g. Pre-K or 3-K)')
('16. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'Social Studies')
('16. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'Science')
('16. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'ELA')
('16. This question is for informational purposes only. It will not be scored and will not be used for accountability for your school. What content areas do you teach? ', 'Math')
('16. This question is for informational purposes only. It wil

Question 16 has 7 options:  
Early childhood (e.g. Pre-K or 3-K), Social Studies, Science, ELA, Math, All subjects (e.g., elementary, self-contained), Other

In [198]:
teacher_2019.loc[:1, teacher_2019.columns.get_level_values(1) == 'N/A'] 

,"20a. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students’ knowledge through content-rich non-fiction.","20b. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for reading and writing experiences grounded in evidence from text, both literary and informational.","20c. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for students to interact with complex grade-level text.","20d. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for students to interact with academic language.","20e. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for focusing deeply on the concepts emphasized in the standards to help students build strong foundations for learning.","20f. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for creating coherent progressions within the standards from previous grades to current grade so student knowledge/skills build onto previous learning as foundations for math concepts.","20g. In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for developing students’ conceptual understanding, procedural fluency, and their ability to apply math in context."
,N/A,N/A,N/A,N/A,N/A,N/A,N/A
0,24.0,24.0,24.0,19.0,18.0,18.0,18.0
1,9.0,9.0,13.0,6.0,9.0,13.0,13.0


In [199]:
for column in teacher_2019.columns:
    if re.match(r'20', column[0]):
        print(column)

('20a.  In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students’ knowledge through content-rich non-fiction.', 'Strongly Disagree')
('20a.  In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students’ knowledge through content-rich non-fiction.', 'Disagree')
('20a.  In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students’ knowledge through content-rich non-fiction.', 'Agree')
('20a.  In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students’ knowledge through content-rich non-fiction.', 'Strongly Agree')
('20a.  In planning my last instructional unit, I had the resources and tools I needed to include multiple opportunities for building students’ knowledge through content-rich non-f

Questions 20s have 5 options:  
Strongly Disagree, Disagree, Agree, Strongly Agree, N/A

In [200]:
teacher_2019.loc[:1, teacher_2019.columns.get_level_values(1) == "I don’t know "] 

,23d. How many adults at this school teach students how to advocate for themselves?,23e. How many adults at this school teach students the organizational skills needed to be prepared for their next level?,23f. How many adults at this school recognize disruptive behavior as social-emotional learning opportunities?,"23g. How many adults at this school teach students the skills they need to regulate their behavior (i.e. by focusing their attention, controlling their emotions, or managing their thinking, behavior, and feelings)?",23h. How many adults at this school have access to school based supports to assist in behavioral/emotional escalations?
,I don’t know,I don’t know,I don’t know,I don’t know,I don’t know
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0


In [201]:
for column in teacher_2019.columns:
    if re.match(r'28', column[0]):
        print(column)

('28a. Adults at this school talk to students about what they plan to do after high school.', 'Strongly disagree')
('28a. Adults at this school talk to students about what they plan to do after high school.', 'Disagree')
('28a. Adults at this school talk to students about what they plan to do after high school.', 'Agree')
('28a. Adults at this school talk to students about what they plan to do after high school.', 'Strongly Agree')
('28a. Adults at this school talk to students about what they plan to do after high school.', "I don't know")
('28b. Adults at this school create an atmosphere that encourages students to continue their education after high school.', 'Strongly disagree')
('28b. Adults at this school create an atmosphere that encourages students to continue their education after high school.', 'Disagree')
('28b. Adults at this school create an atmosphere that encourages students to continue their education after high school.', 'Agree')
('28b. Adults at this school create an a

Questions 2s, 27s, 28 have 5 options:  
Strongly disagree, Disagree, Agree, Strongly agree, I don't know   
Questions 15s have 5 options:   
Very dissatisfied, dissatisfied, Satisfied, Very Satisfied, I don't know  
Questions 23s have 5 options:   
None, Some, A lot, All, I don't know  
The rest of the questions appear to have 4 options

In [202]:
# grab the first part where it has the question number right before the .
questions = teacher_2019.columns.get_level_values(0).unique()[3:]
q_nums= [re.search(r'(\d+[a-z]*)\.', question).group(1) for question in questions]

# start an empty list to store the new column names
new_cols = []
for q_num in q_nums:
    if re.match(r'2[a-z]|15[a-z]|20[a-z]|23[a-z]|27[a-z]|28[a-z]', q_num):
        for i in [1, 2, 3, 4, 0]:
            new_cols.append(q_num + '_' + str(i))
    elif re.match(r'16', q_num):
        for i in range(1, 8):
            new_cols.append(q_num + '_' + str(i))
    else:
        for i in range(1, 5):
            new_cols.append(q_num + '_' + str(i))
            
# append the rest of the column names
new_cols.insert(0, 'parent_response_rate')
new_cols.insert(0, 'school_name')
new_cols.insert(0, 'dbn')

# change column names
teacher_2019.columns = new_cols

In [209]:
teacher_2015.fillna(0, inplace=True)
teacher_2016.fillna(0, inplace=True)
teacher_2017.fillna(0, inplace=True)
teacher_2018.fillna(0, inplace=True)
teacher_2019.fillna(0, inplace=True)

In [210]:
# write data to file
teacher_2015.to_csv('../data/clean_survey/2015_teacher.csv', index=False)
teacher_2016.to_csv('../data/clean_survey/2016_teacher.csv', index=False)
teacher_2017.to_csv('../data/clean_survey/2017_teacher.csv', index=False)
teacher_2018.to_csv('../data/clean_survey/2018_teacher.csv', index=False)
teacher_2019.to_csv('../data/clean_survey/2019_teacher.csv', index=False)

## Test Scores from 2015-2019

In [453]:
# ela scores
ela_scores = pd.read_excel('../data/ela_scores_2013_to_2019.xlsx', sheet_name='All')

In [455]:
# math scores
math_scores = pd.read_excel('../data/math_scores_2013_to_2019.xlsx', sheet_name='All')

In [459]:
ela_scores.head()

,Unnamed: 0,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,% Level 1,# Level 2,% Level 2,# Level 3,% Level 3,# Level 4,% Level 4,# Level 3+4,% Level 3+4
0,01M01532013All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2013,All Students,27,289.296,14,51.8519,11,40.7407,2,7.40741,0,0,2,7.40741
1,01M01532014All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2014,All Students,18,285.111,10,55.5556,8,44.4444,0,0,0,0,0,0
2,01M01532015All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2015,All Students,16,281.812,9,56.25,5,31.25,2,12.5,0,0,2,12.5
3,01M01532016All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2016,All Students,20,292.5,10,50,6,30,4,20,0,0,4,20
4,01M01532017All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2017,All Students,27,302.37,10,37.037,8,29.6296,7,25.9259,2,7.40741,9,33.3333


In [460]:
ela_scores['Grade'].value_counts()

All Grades    7561
3             5258
4             5151
5             5099
6             3338
7             3235
8             3184
Name: Grade, dtype: int64

In [463]:
# I will only be using the data from all grades only, so get rid of the rest
ela_scores = ela_scores[ela_scores['Grade'] == 'All Grades']

In [464]:
ela_scores['Year'].value_counts()

2019    1118
2018    1104
2017    1093
2016    1084
2015    1075
2014    1058
2013    1029
Name: Year, dtype: int64

In [466]:
# I will also only use data from 2015 to 2019
ela_scores = ela_scores[(ela_scores['Year'] != 2013) & (ela_scores['Year'] != 2014)]

In [468]:
# drop first column
ela_scores.drop(columns = ['Unnamed: 0'], inplace=True)

In [470]:
# I only be using mean scores per school, so I will drop most other columns
# Now drop every column after the `Mean Scale Score` column
ela_scores = ela_scores.loc[:, :'Mean Scale Score']

In [472]:
ela_scores['Category'].value_counts()

All Students    5474
Name: Category, dtype: int64

In [477]:
# Drop the `category` column
ela_scores.drop(columns=['Category'], inplace=True)

In [478]:
ela_scores.head(2)

,DBN,School Name,Grade,Year,Number Tested,Mean Scale Score
23,01M015,P.S. 015 ROBERTO CLEMENTE,All Grades,2015,56,283.286
24,01M015,P.S. 015 ROBERTO CLEMENTE,All Grades,2016,51,292.745


In [482]:
# rename columns
col_names = ['dbn', 'school_name', 'grade', 'year', 'number_tested', 'mean_score']

ela_scores.columns = col_names

In [483]:
math_scores.head()

,Unnamed: 0,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,% Level 1,# Level 2,% Level 2,# Level 3,% Level 3,# Level 4,% Level 4,# Level 3+4,% Level 3+4
0,01M01532013All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2013,All Students,27,277.778,16,59.2593,11,40.7407,0,0,0,0,0,0
1,01M01532014All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2014,All Students,18,286.389,6,33.3333,9,50,2,11.1111,1,5.55556,3,16.6667
2,01M01532015All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2015,All Students,17,279.588,10,58.8235,4,23.5294,2,11.7647,1,5.88235,3,17.6471
3,01M01532016All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2016,All Students,21,274.81,13,61.9048,4,19.0476,4,19.0476,0,0,4,19.0476
4,01M01532017All Students,01M015,P.S. 015 ROBERTO CLEMENTE,3,2017,All Students,29,301.552,8,27.5862,9,31.0345,7,24.1379,5,17.2414,12,41.3793


In [484]:
# same as ela scores, only use all grades data and from 2015-2019
math_scores = math_scores[math_scores['Grade'] == 'All Grades']
math_scores = math_scores[(math_scores['Year'] != 2013) & (math_scores['Year'] != 2014)]

In [487]:
# drop unnecessary columns
math_scores.drop(columns=['Unnamed: 0', 'Category'], inplace=True)
math_scores = math_scores.loc[:, :'Mean Scale Score']

In [488]:
math_scores.head(2)

,DBN,School Name,Grade,Year,Number Tested,Mean Scale Score
23,01M015,P.S. 015 ROBERTO CLEMENTE,All Grades,2015,58,278.293
24,01M015,P.S. 015 ROBERTO CLEMENTE,All Grades,2016,52,285.673


In [489]:
# rename columns
col_names = ['dbn', 'school_name', 'grade', 'year', 'number_tested', 'mean_score']

math_scores.columns = col_names

In [492]:
# write to file
# make a directory to place the files in
if not os.path.exists('../data/clean_scores'):
    os.mkdir('../data/clean_scores')

ela_scores.to_csv('../data/clean_scores/ela_scores_2015_to_2019.csv', index=False)
math_scores.to_csv('../data/clean_scores/math_scores_2015_to_2019.csv', index=False)